In [442]:
Zx.<x> = ZZ[]
Qx.<x> = QQ[]
n = 4
sigmamin = 1.277833697
sigmamax =1.8205
q = 12*1024+1

In [443]:
class Node:
    def __init__(self, value):
        self.value = value
        self.leftchild = 0
        self.rightchild = 0
        self.tree = [self.value,self.leftchild,self.rightchild]
        
    def update_tree(self):
        self.tree = [self.value,self.leftchild,self.rightchild]
        
    def __str__(self):
        return '[' + str(self.value) + ',' + str(self.leftchild) + ','  + str(self.rightchild) + ']' 
    
    def __repr__(self):
        return '[' + str(self.value) + ',' + str(self.leftchild) + ','  + str(self.rightchild) + ']'
    
    def print_tree(self, pref=""):
        leaf = "|—————> "
        top = "|_______"
        son1 = "|       "
        son2 = "        "
        width = len(top)

        a = ""
        if (self.value * x^0).degree() and self.leftchild:
            if (pref == ""):
                a += pref + str(self.value) + "\n"
            else:
                a += pref[:-width] + top + str(self.value) + "\n"
            try:
                a += self.leftchild.print_tree(pref + son1)
                a += self.rightchild.print_tree(pref + son2)
            except:
                pass
            return a
        else:
            return (pref[:-width] + leaf + str(self.value) + "\n")
def normalize_tree(tree, sigma):
    if (tree.value * x^0).degree() and tree.leftchild is not None:
        normalize_tree(tree.leftchild, sigma)
        normalize_tree(tree.rightchild, sigma)
    else:
        tree.value = sigma / sqrt(tree.value.real())
        tree.update_tree()

In [444]:
def merge(a,b,n):
    a = a.subs(x=x^2)
    b = x*b.subs(x=x^2)
    return a + b 

In [445]:
def split(f,n):
        f0 = list(f[2*i] for i in range(n//2+1))
        f1 = list(f[2*i+1] for i in range(n//2+1))
        return f.parent()(f0), f.parent()(f1)


In [446]:
def split_fft(f_fft):
    #print("have to split", f_fft)
    try:
        n = len(f_fft)
    except:
        n = 1
    w = roots_dict[n]
    f0_fft = [0] * (n // 2)
    f1_fft = [0] * (n // 2)
    for i in range(n // 2):
        f0_fft[i] = 0.5 * (f_fft[2 * i] + f_fft[2 * i + 1])
        f1_fft[i] = 0.5 * (f_fft[2 * i] - f_fft[2 * i + 1]) * w[2 * i].conjugate()
    return [f0_fft, f1_fft]

In [447]:
def merge_fft(f_list_fft):
    #print("have to merge",f_list_fft)
    f0_fft, f1_fft = f_list_fft
    try:
        n = 2 * len(f0_fft)
    except:
        n = 2
        
    w = roots_dict[n]
    f_fft = [0] * n
    for i in range(n // 2):
        f_fft[2 * i + 0] = f0_fft[i] + w[2 * i] * f1_fft[i]
        f_fft[2 * i + 1] = f0_fft[i] - w[2 * i] * f1_fft[i]
    return f_fft

In [448]:
def add_fft(f, g):
    #print("f+g",f,g)
    assert len(f) == len(g)
    deg = len(f)
    return [f[i] + g[i] for i in range(deg)]

def neg_fft(f):
    deg = len(f)
    return [- f[i] for i in range(deg)]

def sub_fft(f, g):
    return add_fft(f, neg_fft(g))

def mul_fft(f_fft, g_fft):
    deg = len(f_fft)
    return [f_fft[i] * g_fft[i] for i in range(deg)]

def div_fft(f_fft, g_fft):
    assert len(f_fft) == len(g_fft)
    deg = len(f_fft)
    return [f_fft[i] / g_fft[i] for i in range(deg)]


In [449]:
def CyclicRotate(input, n):
    return input[-n:] + input[0:-n]

In [450]:
def HermitianAdjointPoly(p, n):
    f=[p[0]]
    for i in range(1,n):    
        f.append(-p[n-i])
    return p.parent()(f) 

In [451]:
def make_ok(matrix):
    l1 = matrix[:4]
    for row in matrix[4:]:
        newr = []
        for el in row:
            if '/' in str(el):
                n,d = str(el).split('/')
                newr.append(round(float(float(n)/float(d)),2))
            else:
                newr.append(el)
        l1.append(newr)
    return l1

In [452]:
def FFT(f, n):
    #print(f)
    if (n > 2):
        f0, f1 = split(f,n)
        f0_fft = FFT(f0,n//2)
        f1_fft = FFT(f1,n//2)
        f_fft = merge_fft([f0_fft, f1_fft])
    elif (n == 2):
        f_fft = [0] * n
        f_fft[0] = f[0] + i * f[1]
        f_fft[1] = f[0] - i * f[1]
    return f_fft

In [453]:
def invFFT2(f_fft):
    try:
        n = len(f_fft)
    except:
        n = 1
        f = f_fft
    f = [0] * n
    if (n > 2):
        f0_fft, f1_fft = split_fft(f_fft)
        f0 = invFFT2(f0_fft)
        f1 = invFFT2(f1_fft)
        f = merge(f0, f1,n)
    elif (n == 2):
        
        f[0] = f_fft[0].real()
        f[1] = f_fft[0].imag()
    return Qx(f)

In [454]:
def LatticeToPoly4(M, n): #!!!!!!!!!!!!! author postulga
    p00= M[0][:n]
    p01= M[0][n:]
    p10= M[n][:n]
    p11= M[n][n:]
    return p00, p01, p10, p11

In [455]:
def PolyToLattice4(p00, p01, p10, p11, n):
    M=[]
    p=[p00.coefficients(sparse=False),p01.coefficients(sparse=False),p10.coefficients(sparse=False),p11.coefficients(sparse=False)]
    for i in range(4):
        while len(p[i])!=n:
            p[i].append(0)
    for i in range(n):
        m1 = CyclicRotate(p[0], i)
        m2 = CyclicRotate(p[1], i)
        M.append(m1+m2)
    for i in range(n):
        m1 = CyclicRotate(p[2], i)
        m2 = CyclicRotate(p[3], i)
        M.append(m1+m2)
    return M

In [456]:
from os import urandom

LN2 = 0.69314718056

#difference!
def UniformBits(k):
    return int.from_bytes(bytes(list(floor(uniform(0, 256)) for i in range(k / 8))), 'big')

def BaseSampler():
    u = UniformBits(72) #difference!
    z_0 = 0
    RCDT = [3024686241123004913666, 1564742784480091954050, 636254429462080897535, 199560484645026482916, 47667343854657281903, 8595902006365044063, 1163297957344668388, 117656387352093658, 8867391802663976, 496969357462633, 20680885154299, 638331848991, 14602316184, 247426747, 3104126, 28824, 198, 1, 0]
    for i in range(0, 18):
        z_0 = z_0 + int(u<RCDT[i]) 
    return z_0

def ApproxExp(x, ccs):
    C = [0x00000004741183A3,0x00000036548CFC06,0x0000024FDCBF140A,0x0000171D939DE045,0x0000D00CF58F6F84,0x000680681CF796E3,0x002D82D8305B0FEA,0x011111110E066FD0,0x0555555555070F00,0x155555555581FF00,0x400000000002B400,0x7FFFFFFFFFFF4800,0x8000000000000000]
    y = C[0]
    z = floor(2^63*x)
    for i in range(1, 13):
        y = C[i] - (z*y) >> 63
    z = floor(2^63*ccs)
    y = (z*y) >> 63
    return y

def BerExp(x, ccs):
    s = floor(x/LN2) #difference!
    r = x - s*LN2 #difference!
    s = min(s, 63)
    z = (2*ApproxExp(r, ccs) - 1) >> s
    for i in range(56, -8, -8):
        p = UniformBits(8) #difference!
        w = p - ((z >> i) & 0xFF)
        if int(w) == 0:
            break
    return int(w < 0)

def SamplerZ(mu, sigma, sigmamin, sigmamax):
    r = mu - int(floor(mu)) #difference!
    ccs = sigmamin/sigma
    while True:
        z_0 = BaseSampler()
        b = UniformBits(8)&0x1 #difference!
        z = b + (2*b-1)*z_0
        x = (z-r)^2/2/sigma^2 - z_0^2/2/sigmamax^2
        if BerExp(x, ccs) == 1:
            return z + int(floor(mu)) #difference!


In [457]:
def LDL(g00, g01, g10, g11, n):
    D00 = g00
    g100 = FFT(g10,n)
    g000 = FFT(g00,n)
    L10 = div_fft(g100,g000)
    L10_star = HermitianAdjointPoly(invFFT2(L10),n)
    D11 = sub_fft(FFT(g11,n), mul_fft(mul_fft(L10,FFT(L10_star,n)),g000))
    L = PolyToLattice4(1*x^0,0*x^0,Qx(invFFT2(L10)),1*x^0,n)
    D = PolyToLattice4(D00,0*x^0,0*x^0,Qx(invFFT2(D11)),n)
    return L,D

In [458]:
f = 244*x^3 + 353*x^2 + 320*x + 296
g =  406*x^3 + 237*x^2 + 462*x + 321
F =  -185*x^3 - 101*x^2 + 152*x + 138
G = -279*x^3 - 54*x^2 + 161*x + 119

In [459]:
l,d = LDL(g,-f,G,-F,n)

In [460]:
def ffLDL(g00, g01, g10, g11, n):
    L,D = LDL(Qx(g00), Qx(g01), Qx(g10), Qx(g11), n)
    L10 = Qx(LatticeToPoly4(L,n)[2])
    tree = Node(L10)
    D00 = Qx(LatticeToPoly4(D,n)[0])
    D11 = Qx(LatticeToPoly4(D,n)[3])
    if n == 2:
        tree.leftchild = FFT(D00,n)[0]
        tree.rightchild = FFT(D11,n)[0]
        tree.update_tree()
        #print(tree)
        return tree
    else:
        d00,d01 = split_fft(FFT(D00,n))
        d10,d11 = split_fft(FFT(D11,n))
        d01_star = HermitianAdjointPoly(invFFT2(d01),n//2)
        d11_star = HermitianAdjointPoly(invFFT2(d00),n//2)
        tree.leftchild = ffLDL(invFFT2(d00),invFFT2(d01),d01_star,invFFT2(d00),n//2)
        tree.rightchild = ffLDL(invFFT2(d01),invFFT2(d11),d11_star,invFFT2(d10),n//2)
    return tree   
           

In [461]:
T = ffLDL(g,-f,G,-F,n)

In [462]:
T

[-3710441/19576182*x^3 - 34169051/88007968*x^2 + 18066663/358357105*x - 63913423/511455876,[-7994/5307*x + 1736/5307,237*I + 321,-1730653/5307*I - 2344049/5307],[-1713/2702*x + 186/1351,406*I + 462,-446589759205/3206501228*I - 14083074733953/35248322242]]

In [491]:
def ffSampling(t0_, t1_, T, sigmamin, sigmamax, q, n):
    # _ means constant , __ means ' 
    if n == 1:
        #print('t0 t1',t0_,t1_)
        #print('TT',T,n)
        sigma = T
        #print("mu1", t0_[0].real())
        #print("sigma",float(sigma.real()))
        #print("mu2,", t1_[0].real())
        
        z0_ = SamplerZ(t0_[0].real(),float(sigma.real()),sigmamin, sigmamax)
        z1_ = SamplerZ(t1_[0].real(),float(sigma.real()),sigmamin, sigmamax)
        #print('SAMPLERZ',z0_,z1_)
        #print("==================")
        return [z0_],[z1_]
    #print('T',T,n)
    l = T.value
    T0 = T.leftchild
    T1 = T.rightchild
    t1 = split_fft(t1_)
    z1 = ffSampling(*t1, T1, sigmamin, sigmamax, q, n/2)
    z1_ = merge_fft(z1)
    t0__ = add_fft(t0_, mul_fft(sub_fft(t1_,z1_),l))
    t0 = split_fft(t0__)
    z0 = ffSampling(*t0, T0, sigmamin, sigmamax, q, n/2)
    z0_ = merge_fft(z0)
    return z0_,z1_

In [534]:
p1 = 4*x^3 + x^2 + x + 1
p2 = 2*x^3 + x + 2
f = 244*x^3 + 353*x^2 + 320*x + 296
g =  406*x^3 + 237*x^2 + 462*x + 321
F =  -185*x^3 - 101*x^2 + 152*x + 138
G = -279*x^3 - 54*x^2 + 161*x + 119

In [484]:
p1fft = FFT(p1,n)
p2fft = FFT(p2,n)

In [488]:
ffSampling(p1fft,p2fft,T,sigmamin, sigmamax, q,4)

([4.00000000000000 + 4.65685424949238*I,
  4.00000000000000 - 6.65685424949238*I,
  4.00000000000000 - 4.65685424949238*I,
  4.00000000000000 + 6.65685424949238*I],
 [-0.414213562373095 + 9.99200722162641e-16*I,
  2.41421356237309 - 9.99200722162641e-16*I,
  -0.414213562373095 - 9.99200722162641e-16*I,
  2.41421356237309 + 9.99200722162641e-16*I])

In [528]:
def InnerProduct(a,b,n):
    # return sum([a[i] * b[i] for i in range(n)])
    s=0
    for i in range(n):
        s=s+a[i]*b[i]
    return(s)

# formula 3.9 from page 24, author: Evgen Postulga
def EuclideanNorm(a,n):
    # return sqrt(InnerProduct(a,a,n))
    b=InnerProduct(a,a,n)
    return b^(1/2)

# formula 3.25 from page 30, author: Maxim Pushkar
def FieldNorm(f, n):
    f0, f1 = Split(f, n)
    iks = f.parent()([0, 1])
    return (f0^2 - iks * f1^2) % (iks^(n/2)+1)
    # return (f0^2-x*f1^2)%(x^(n/2)+1)

# NNT representation of polynomial f from Zq[x]/x^n+1, page 28, author Karina Ilchenko
def NTT(f, n, q):
    # Zp
    #g = f.change_ring(Integers(q))
    #return list(g(root[0]) for root in (x^n+1).roots(Integers(q)))    
    roots = (x^n + 1).roots(Integers(q))
    ans = [f.subs(x = i[0]) % q for i in roots]
    return ans

In [520]:
from Crypto.Hash import SHAKE256
def HashToPoint(noise, message, q, n):
    k = floor(2^16 / q)
    c = n*[0]
    ctx = SHAKE256.new()
    ctx.update(noise)
    ctx.update(message)
    i = 0
    while i < n:
        t = int.from_bytes(ctx.read(2), byteorder='big')
        if t < k * q:
            c[i] = t % q
            i += 1
    return Zx(c)


In [496]:
def PolyToLattice1(p00, p01, p10, p11, n):
    M=[]
    p=[p00,p01,p10,p11]
    for i in range(4):
        while len(p[i])!=n:
            p[i].append(0)
    for i in range(n):
        m1 = CyclicRotate(p[0], i)
        m2 = CyclicRotate(p[1], i)
        M.append(m1+m2)
    for i in range(n):
        m1 = CyclicRotate(p[2], i)
        m2 = CyclicRotate(p[3], i)
        M.append(m1+m2)
    return M

In [502]:
B = Matrix(PolyToLattice1(FFT(g,4),neg_fft(FFT(f,4)),FFT(G,4),neg_fft(FFT(F,4)),4))

In [545]:
def Balance(f,q,n):
    g = list(((f[i] + q//2) % q) - q//2 for i in range(n))
    return f.parent()(g)

# formula 3.21 from page 28, author: Evgen Postulga
def Split(f,n):
    f0 = list(f[2*i+0] for i in range(n//2))
    f1 = list(f[2*i+1] for i in range(n//2))
    return f.parent()(f0), f.parent()(f1)  

# formula 3.22 from page 28, author: Evgen Postulga
def Merge(a,b):
    a = a.subs(x=x^2)
    b = x*b.subs(x=x^2)
    return a+b

In [530]:
def Sign(message, B, T, beta, sigma_min, sigma_max, q, n): 
    # bound = int(beta^2)
    #r = UniformBits(320)
    r = b'befhehfjfn'
    c = HashToPoint(r, message, q, n)
    
    Qy.<y> = PolynomialRing(QQ)
    Qphi.<Y> = Qy.quotient(y^n+1)
    
    g, f, G, F = LatticeToPoly4(B, 4)  # наверное
    f, F = neg_fft(f), neg_fft(F)
    
    t0 = mul_fft(FFT(Qphi(c.subs(x=y)),n), f) 
    t1 = mul_fft(FFT(Qphi(c.subs(x=y)),n), f)  #todo
    
    t = vector(t0 + t1)
    
    # альтернативная версия + сверить обе
    
    while True:
        z1,z2 = ffSampling(t0, t1, T, sigma_min, sigma_max, q, n)
        s = (t - vector(z1+z2)) * B
        
        if EuclideanNorm(s,2*n) <= beta:
            s1, s2 = invFFT2(s[:n]), invFFT2(s[n:])
            # s = Compress(s2, ...)
            s = s2
            return r, s

In [526]:
beta = sqrt(34034726)

In [531]:
Sign(b'hello',B,T,beta,sigmamin,sigmamax,q,n)

(b'befhehfjfn',
 -729569125/1716889*x^3 + 2376274429/5592067*x + 3663905785/2359318)

In [543]:
def Verify(message, r, s, pk, beta, q, n):
    bound = int(beta^2)
    c = HashToPoint(r, message, q, n)
    # s2 = Decompress(s, ...)
    s2 = s
    
    # s2 ∈ Z[x]/(x^n + 1)
    #s2 = Qx(s2)
    
    h = pk
    s1 = Balance((c - s2*h) % (x^n + 1), q, n)
    
    if (EuclideanNorm(s1,n)^2 + EuclideanNorm(s2,n)^2) <= bound:
        print("signature accepted")
        return True
    
    print("signature rejected")
    return False

In [541]:
s = -729569125/1716889*x^3 + 2376274429/5592067*x + 3663905785/2359318

In [547]:
print(Verify(b'not hello',b'befhehfjfn',s,f_q,beta,q,n))

signature rejected
False


In [535]:
g*(1/f)

(203/122*x^3 + 237/244*x^2 + 231/122*x + 321/244)/(x^3 + 353/244*x^2 + 80/61*x + 74/61)

In [537]:
tt = Zx.change_ring(Integers(q)).quotient(x^4+1)
f_q = Zx(lift(tt(g) / tt(f))) 

In [481]:
def NTRUGen(q, n):
    pass

In [317]:
# f = 244*x^3 + 353*x^2 + 320*x + 296
# g =  406*x^3 + 237*x^2 + 462*x + 321
# F =  -185*x^3 - 101*x^2 + 152*x + 138
# G = -279*x^3 - 54*x^2 + 161*x + 119
# g1 = FFT(g,n)
# f1 = FFT(f,n)
# G1 = FFT(G,n)
# F1 = FFT(F,n)
# print(f1,g1,F1,G1)
# B = [[g1,neg_fft(f1)],[G1,neg_fft(F1)]]
        

In [467]:
def KeyGenFFT(sigma, q, n):
    #f,g,F,G = NTRUGen(q, n)
    f = 244*x^3 + 353*x^2 + 320*x + 296
    g =  406*x^3 + 237*x^2 + 462*x + 321
    F =  -185*x^3 - 101*x^2 + 152*x + 138
    G = -279*x^3 - 54*x^2 + 161*x + 119
    g1 = FFT(g,n)
    f1 = FFT(f,n)
    G1 = FFT(G,n)
    F1 = FFT(F,n)
    B = [[g1,neg_fft(f1)],[G1,neg_fft(F1)]]
    g2 = HermitianAdjointPoly(g1,n)
    f2 = HermitianAdjointPoly(f1,n)
    G2 = HermitianAdjointPoly(G1,n)
    F2 = HermitianAdjointPoly(F1,n)
    B1 = [[g1,G1],[neg_fft(f1),neg_fft(F1)]]
    GG = []
    print(GG)

In [319]:
KeyGenFFT(1,1,4)

[]


In [321]:
j = i

"""
Constants (complex roots of cyclotomic polynomials) for the NTT.
These were computed using scripts/generate_constants.sage.
If this module is loaded with fft_short, we only need the first half of each phi*_roots table.
"""


""" The roots of phi_4 = x^2 + 1
The second root is the conjugates of the first one.
"""
phi4_roots = [1.00000000000000j, -1.00000000000000j]

""" The roots of phi_8 = x^4 + 1
The second half of the roots are is the conjugates of the first half.
The root at index (2 * i + 1) is the negation of the root at index (2 * i).
"""
phi8_roots = [0.707106781186548 + 0.707106781186547j,
              -0.707106781186548 - 0.707106781186547j,
              0.707106781186548 - 0.707106781186547j,
              -0.707106781186548 + 0.707106781186547j]


""" The roots of phi_16 = x^8 + 1
The second half of the roots are is the conjugates of the first half.
The root at index (2 * i + 1) is the negation of the root at index (2 * i).
"""
phi16_roots = [0.923879532511287 + 0.382683432365090j,
               -0.923879532511287 - 0.382683432365090j,
               0.382683432365090 - 0.923879532511287j,
               -0.382683432365090 + 0.923879532511287j,
               0.923879532511287 - 0.382683432365090j,
               -0.923879532511287 + 0.382683432365090j,
               0.382683432365090 + 0.923879532511287j,
               -0.382683432365090 - 0.923879532511287j]


""" The roots of phi_32 = x^16 + 1
The second half of the roots are is the conjugates of the first half.
The root at index (2 * i + 1) is the negation of the root at index (2 * i).
"""
phi32_roots = [0.980785280403230 + 0.195090322016128j,
               -0.980785280403230 - 0.195090322016128j,
               0.195090322016128 - 0.980785280403230j,
               -0.195090322016128 + 0.980785280403230j,
               0.831469612302545 - 0.555570233019602j,
               -0.831469612302545 + 0.555570233019602j,
               0.555570233019602 + 0.831469612302545j,
               -0.555570233019602 - 0.831469612302545j,
               0.980785280403230 - 0.195090322016128j,
               -0.980785280403230 + 0.195090322016128j,
               0.195090322016128 + 0.980785280403230j,
               -0.195090322016128 - 0.980785280403230j,
               0.831469612302545 + 0.555570233019602j,
               -0.831469612302545 - 0.555570233019602j,
               0.555570233019602 - 0.831469612302545j,
               -0.555570233019602 + 0.831469612302545j]


""" The roots of phi_64 = x^32 + 1
The second half of the roots are is the conjugates of the first half.
The root at index (2 * i + 1) is the negation of the root at index (2 * i).
"""
phi64_roots = [0.995184726672197 + 0.0980171403295606j,
               -0.995184726672197 - 0.0980171403295606j,
               0.0980171403295606 - 0.995184726672197j,
               -0.0980171403295606 + 0.995184726672197j,
               0.773010453362737 - 0.634393284163646j,
               -0.773010453362737 + 0.634393284163646j,
               0.634393284163645 + 0.773010453362737j,
               -0.634393284163645 - 0.773010453362737j,
               0.956940335732209 - 0.290284677254462j,
               -0.956940335732209 + 0.290284677254462j,
               0.290284677254462 + 0.956940335732209j,
               -0.290284677254462 - 0.956940335732209j,
               0.881921264348355 + 0.471396736825998j,
               -0.881921264348355 - 0.471396736825998j,
               0.471396736825998 - 0.881921264348355j,
               -0.471396736825998 + 0.881921264348355j,
               0.995184726672197 - 0.0980171403295606j,
               -0.995184726672197 + 0.0980171403295606j,
               0.0980171403295606 + 0.995184726672197j,
               -0.0980171403295606 - 0.995184726672197j,
               0.773010453362737 + 0.634393284163646j,
               -0.773010453362737 - 0.634393284163646j,
               0.634393284163645 - 0.773010453362737j,
               -0.634393284163645 + 0.773010453362737j,
               0.956940335732209 + 0.290284677254462j,
               -0.956940335732209 - 0.290284677254462j,
               0.290284677254462 - 0.956940335732209j,
               -0.290284677254462 + 0.956940335732209j,
               0.881921264348355 - 0.471396736825998j,
               -0.881921264348355 + 0.471396736825998j,
               0.471396736825998 + 0.881921264348355j,
               -0.471396736825998 - 0.881921264348355j]


""" The roots of phi_128 = x^64 + 1
The second half of the roots are is the conjugates of the first half.
The root at index (2 * i + 1) is the negation of the root at index (2 * i).
"""
phi128_roots = [0.998795456205172 + 0.0490676743274180j,
                -0.998795456205172 - 0.0490676743274180j,
                0.0490676743274180 - 0.998795456205172j,
                -0.0490676743274180 + 0.998795456205172j,
                0.740951125354959 - 0.671558954847018j,
                -0.740951125354959 + 0.671558954847018j,
                0.671558954847018 + 0.740951125354959j,
                -0.671558954847018 - 0.740951125354959j,
                0.941544065183021 - 0.336889853392220j,
                -0.941544065183021 + 0.336889853392220j,
                0.336889853392220 + 0.941544065183021j,
                -0.336889853392220 - 0.941544065183021j,
                0.903989293123443 + 0.427555093430282j,
                -0.903989293123443 - 0.427555093430282j,
                0.427555093430282 - 0.903989293123443j,
                -0.427555093430282 + 0.903989293123443j,
                0.989176509964781 - 0.146730474455362j,
                -0.989176509964781 + 0.146730474455362j,
                0.146730474455362 + 0.989176509964781j,
                -0.146730474455362 - 0.989176509964781j,
                0.803207531480645 + 0.595699304492433j,
                -0.803207531480645 - 0.595699304492433j,
                0.595699304492433 - 0.803207531480645j,
                -0.595699304492433 + 0.803207531480645j,
                0.970031253194544 + 0.242980179903264j,
                -0.970031253194544 - 0.242980179903264j,
                0.242980179903264 - 0.970031253194544j,
                -0.242980179903264 + 0.970031253194544j,
                0.857728610000272 - 0.514102744193222j,
                -0.857728610000272 + 0.514102744193222j,
                0.514102744193222 + 0.857728610000272j,
                -0.514102744193222 - 0.857728610000272j,
                0.998795456205172 - 0.0490676743274180j,
                -0.998795456205172 + 0.0490676743274180j,
                0.0490676743274180 + 0.998795456205172j,
                -0.0490676743274180 - 0.998795456205172j,
                0.740951125354959 + 0.671558954847018j,
                -0.740951125354959 - 0.671558954847018j,
                0.671558954847018 - 0.740951125354959j,
                -0.671558954847018 + 0.740951125354959j,
                0.941544065183021 + 0.336889853392220j,
                -0.941544065183021 - 0.336889853392220j,
                0.336889853392220 - 0.941544065183021j,
                -0.336889853392220 + 0.941544065183021j,
                0.903989293123443 - 0.427555093430282j,
                -0.903989293123443 + 0.427555093430282j,
                0.427555093430282 + 0.903989293123443j,
                -0.427555093430282 - 0.903989293123443j,
                0.989176509964781 + 0.146730474455362j,
                -0.989176509964781 - 0.146730474455362j,
                0.146730474455362 - 0.989176509964781j,
                -0.146730474455362 + 0.989176509964781j,
                0.803207531480645 - 0.595699304492433j,
                -0.803207531480645 + 0.595699304492433j,
                0.595699304492433 + 0.803207531480645j,
                -0.595699304492433 - 0.803207531480645j,
                0.970031253194544 - 0.242980179903264j,
                -0.970031253194544 + 0.242980179903264j,
                0.242980179903264 + 0.970031253194544j,
                -0.242980179903264 - 0.970031253194544j,
                0.857728610000272 + 0.514102744193222j,
                -0.857728610000272 - 0.514102744193222j,
                0.514102744193222 - 0.857728610000272j,
                -0.514102744193222 + 0.857728610000272j]


""" The roots of phi_256 = x^128 + 1
The second half of the roots are is the conjugates of the first half.
The root at index (2 * i + 1) is the negation of the root at index (2 * i).
"""
phi256_roots = [0.999698818696204 + 0.0245412285229123j,
                -0.999698818696204 - 0.0245412285229123j,
                0.0245412285229123 - 0.999698818696204j,
                -0.0245412285229123 + 0.999698818696204j,
                0.724247082951467 - 0.689540544737067j,
                -0.724247082951467 + 0.689540544737067j,
                0.689540544737067 + 0.724247082951467j,
                -0.689540544737067 - 0.724247082951467j,
                0.932992798834739 - 0.359895036534988j,
                -0.932992798834739 + 0.359895036534988j,
                0.359895036534988 + 0.932992798834739j,
                -0.359895036534988 - 0.932992798834739j,
                0.914209755703531 + 0.405241314004990j,
                -0.914209755703531 - 0.405241314004990j,
                0.405241314004990 - 0.914209755703531j,
                -0.405241314004990 + 0.914209755703531j,
                0.985277642388941 - 0.170961888760301j,
                -0.985277642388941 + 0.170961888760301j,
                0.170961888760301 + 0.985277642388941j,
                -0.170961888760301 - 0.985277642388941j,
                0.817584813151584 + 0.575808191417845j,
                -0.817584813151584 - 0.575808191417845j,
                0.575808191417845 - 0.817584813151584j,
                -0.575808191417845 + 0.817584813151584j,
                0.975702130038529 + 0.219101240156870j,
                -0.975702130038529 - 0.219101240156870j,
                0.219101240156870 - 0.975702130038529j,
                -0.219101240156870 + 0.975702130038529j,
                0.844853565249707 - 0.534997619887097j,
                -0.844853565249707 + 0.534997619887097j,
                0.534997619887097 + 0.844853565249707j,
                -0.534997619887097 - 0.844853565249707j,
                0.997290456678690 - 0.0735645635996674j,
                -0.997290456678690 + 0.0735645635996674j,
                0.0735645635996674 + 0.997290456678690j,
                -0.0735645635996674 - 0.997290456678690j,
                0.757208846506485 + 0.653172842953777j,
                -0.757208846506485 - 0.653172842953777j,
                0.653172842953777 - 0.757208846506485j,
                -0.653172842953777 + 0.757208846506485j,
                0.949528180593037 + 0.313681740398892j,
                -0.949528180593037 - 0.313681740398892j,
                0.313681740398892 - 0.949528180593037j,
                -0.313681740398892 + 0.949528180593037j,
                0.893224301195515 - 0.449611329654607j,
                -0.893224301195515 + 0.449611329654607j,
                0.449611329654607 + 0.893224301195515j,
                -0.449611329654607 - 0.893224301195515j,
                0.992479534598710 + 0.122410675199216j,
                -0.992479534598710 - 0.122410675199216j,
                0.122410675199216 - 0.992479534598710j,
                -0.122410675199216 + 0.992479534598710j,
                0.788346427626606 - 0.615231590580627j,
                -0.788346427626606 + 0.615231590580627j,
                0.615231590580627 + 0.788346427626606j,
                -0.615231590580627 - 0.788346427626606j,
                0.963776065795440 - 0.266712757474898j,
                -0.963776065795440 + 0.266712757474898j,
                0.266712757474898 + 0.963776065795440j,
                -0.266712757474898 - 0.963776065795440j,
                0.870086991108711 + 0.492898192229784j,
                -0.870086991108711 - 0.492898192229784j,
                0.492898192229784 - 0.870086991108711j,
                -0.492898192229784 + 0.870086991108711j,
                0.999698818696204 - 0.0245412285229123j,
                -0.999698818696204 + 0.0245412285229123j,
                0.0245412285229123 + 0.999698818696204j,
                -0.0245412285229123 - 0.999698818696204j,
                0.724247082951467 + 0.689540544737067j,
                -0.724247082951467 - 0.689540544737067j,
                0.689540544737067 - 0.724247082951467j,
                -0.689540544737067 + 0.724247082951467j,
                0.932992798834739 + 0.359895036534988j,
                -0.932992798834739 - 0.359895036534988j,
                0.359895036534988 - 0.932992798834739j,
                -0.359895036534988 + 0.932992798834739j,
                0.914209755703531 - 0.405241314004990j,
                -0.914209755703531 + 0.405241314004990j,
                0.405241314004990 + 0.914209755703531j,
                -0.405241314004990 - 0.914209755703531j,
                0.985277642388941 + 0.170961888760301j,
                -0.985277642388941 - 0.170961888760301j,
                0.170961888760301 - 0.985277642388941j,
                -0.170961888760301 + 0.985277642388941j,
                0.817584813151584 - 0.575808191417845j,
                -0.817584813151584 + 0.575808191417845j,
                0.575808191417845 + 0.817584813151584j,
                -0.575808191417845 - 0.817584813151584j,
                0.975702130038529 - 0.219101240156870j,
                -0.975702130038529 + 0.219101240156870j,
                0.219101240156870 + 0.975702130038529j,
                -0.219101240156870 - 0.975702130038529j,
                0.844853565249707 + 0.534997619887097j,
                -0.844853565249707 - 0.534997619887097j,
                0.534997619887097 - 0.844853565249707j,
                -0.534997619887097 + 0.844853565249707j,
                0.997290456678690 + 0.0735645635996674j,
                -0.997290456678690 - 0.0735645635996674j,
                0.0735645635996674 - 0.997290456678690j,
                -0.0735645635996674 + 0.997290456678690j,
                0.757208846506485 - 0.653172842953777j,
                -0.757208846506485 + 0.653172842953777j,
                0.653172842953777 + 0.757208846506485j,
                -0.653172842953777 - 0.757208846506485j,
                0.949528180593037 - 0.313681740398892j,
                -0.949528180593037 + 0.313681740398892j,
                0.313681740398892 + 0.949528180593037j,
                -0.313681740398892 - 0.949528180593037j,
                0.893224301195515 + 0.449611329654607j,
                -0.893224301195515 - 0.449611329654607j,
                0.449611329654607 - 0.893224301195515j,
                -0.449611329654607 + 0.893224301195515j,
                0.992479534598710 - 0.122410675199216j,
                -0.992479534598710 + 0.122410675199216j,
                0.122410675199216 + 0.992479534598710j,
                -0.122410675199216 - 0.992479534598710j,
                0.788346427626606 + 0.615231590580627j,
                -0.788346427626606 - 0.615231590580627j,
                0.615231590580627 - 0.788346427626606j,
                -0.615231590580627 + 0.788346427626606j,
                0.963776065795440 + 0.266712757474898j,
                -0.963776065795440 - 0.266712757474898j,
                0.266712757474898 - 0.963776065795440j,
                -0.266712757474898 + 0.963776065795440j,
                0.870086991108711 - 0.492898192229784j,
                -0.870086991108711 + 0.492898192229784j,
                0.492898192229784 + 0.870086991108711j,
                -0.492898192229784 - 0.870086991108711j]


""" The roots of phi_512 = x^256 + 1
The second half of the roots are is the conjugates of the first half.
The root at index (2 * i + 1) is the negation of the root at index (2 * i).
"""
phi512_roots = [0.999924701839145 + 0.0122715382857199j,
                -0.999924701839145 - 0.0122715382857199j,
                0.0122715382857199 - 0.999924701839145j,
                -0.0122715382857199 + 0.999924701839145j,
                0.715730825283819 - 0.698376249408973j,
                -0.715730825283819 + 0.698376249408973j,
                0.698376249408973 + 0.715730825283819j,
                -0.698376249408973 - 0.715730825283819j,
                0.928506080473216 - 0.371317193951837j,
                -0.928506080473216 + 0.371317193951837j,
                0.371317193951837 + 0.928506080473216j,
                -0.371317193951837 - 0.928506080473216j,
                0.919113851690058 + 0.393992040061048j,
                -0.919113851690058 - 0.393992040061048j,
                0.393992040061048 - 0.919113851690058j,
                -0.393992040061048 + 0.919113851690058j,
                0.983105487431216 - 0.183039887955141j,
                -0.983105487431216 + 0.183039887955141j,
                0.183039887955141 + 0.983105487431216j,
                -0.183039887955141 - 0.983105487431216j,
                0.824589302785025 + 0.565731810783613j,
                -0.824589302785025 - 0.565731810783613j,
                0.565731810783613 - 0.824589302785025j,
                -0.565731810783613 + 0.824589302785025j,
                0.978317370719628 + 0.207111376192219j,
                -0.978317370719628 - 0.207111376192219j,
                0.207111376192219 - 0.978317370719628j,
                -0.207111376192219 + 0.978317370719628j,
                0.838224705554838 - 0.545324988422046j,
                -0.838224705554838 + 0.545324988422046j,
                0.545324988422046 + 0.838224705554838j,
                -0.545324988422046 - 0.838224705554838j,
                0.996312612182778 - 0.0857973123444399j,
                -0.996312612182778 + 0.0857973123444399j,
                0.0857973123444399 + 0.996312612182778j,
                -0.0857973123444399 - 0.996312612182778j,
                0.765167265622459 + 0.643831542889791j,
                -0.765167265622459 - 0.643831542889791j,
                0.643831542889791 - 0.765167265622459j,
                -0.643831542889791 + 0.765167265622459j,
                0.953306040354194 + 0.302005949319228j,
                -0.953306040354194 - 0.302005949319228j,
                0.302005949319228 - 0.953306040354194j,
                -0.302005949319228 + 0.953306040354194j,
                0.887639620402854 - 0.460538710958240j,
                -0.887639620402854 + 0.460538710958240j,
                0.460538710958240 + 0.887639620402854j,
                -0.460538710958240 - 0.887639620402854j,
                0.993906970002356 + 0.110222207293883j,
                -0.993906970002356 - 0.110222207293883j,
                0.110222207293883 - 0.993906970002356j,
                -0.110222207293883 + 0.993906970002356j,
                0.780737228572094 - 0.624859488142386j,
                -0.780737228572094 + 0.624859488142386j,
                0.624859488142386 + 0.780737228572094j,
                -0.624859488142386 - 0.780737228572094j,
                0.960430519415566 - 0.278519689385053j,
                -0.960430519415566 + 0.278519689385053j,
                0.278519689385053 + 0.960430519415566j,
                -0.278519689385053 - 0.960430519415566j,
                0.876070094195407 + 0.482183772079123j,
                -0.876070094195407 - 0.482183772079123j,
                0.482183772079123 - 0.876070094195407j,
                -0.482183772079123 + 0.876070094195407j,
                0.999322384588350 - 0.0368072229413588j,
                -0.999322384588350 + 0.0368072229413588j,
                0.0368072229413588 + 0.999322384588350j,
                -0.0368072229413588 - 0.999322384588350j,
                0.732654271672413 + 0.680600997795453j,
                -0.732654271672413 - 0.680600997795453j,
                0.680600997795453 - 0.732654271672413j,
                -0.680600997795453 + 0.732654271672413j,
                0.937339011912575 + 0.348418680249435j,
                -0.937339011912575 - 0.348418680249435j,
                0.348418680249435 - 0.937339011912575j,
                -0.348418680249435 + 0.937339011912575j,
                0.909167983090522 - 0.416429560097637j,
                -0.909167983090522 + 0.416429560097637j,
                0.416429560097637 + 0.909167983090522j,
                -0.416429560097637 - 0.909167983090522j,
                0.987301418157858 + 0.158858143333861j,
                -0.987301418157858 - 0.158858143333861j,
                0.158858143333861 - 0.987301418157858j,
                -0.158858143333861 + 0.987301418157858j,
                0.810457198252595 - 0.585797857456439j,
                -0.810457198252595 + 0.585797857456439j,
                0.585797857456439 + 0.810457198252595j,
                -0.585797857456439 - 0.810457198252595j,
                0.972939952205560 - 0.231058108280671j,
                -0.972939952205560 + 0.231058108280671j,
                0.231058108280671 + 0.972939952205560j,
                -0.231058108280671 - 0.972939952205560j,
                0.851355193105265 + 0.524589682678469j,
                -0.851355193105265 - 0.524589682678469j,
                0.524589682678469 - 0.851355193105265j,
                -0.524589682678469 + 0.851355193105265j,
                0.998118112900149 + 0.0613207363022086j,
                -0.998118112900149 - 0.0613207363022086j,
                0.0613207363022086 - 0.998118112900149j,
                -0.0613207363022086 + 0.998118112900149j,
                0.749136394523459 - 0.662415777590172j,
                -0.749136394523459 + 0.662415777590172j,
                0.662415777590172 + 0.749136394523459j,
                -0.662415777590172 - 0.749136394523459j,
                0.945607325380521 - 0.325310292162263j,
                -0.945607325380521 + 0.325310292162263j,
                0.325310292162263 + 0.945607325380521j,
                -0.325310292162263 - 0.945607325380521j,
                0.898674465693954 + 0.438616238538528j,
                -0.898674465693954 - 0.438616238538528j,
                0.438616238538528 - 0.898674465693954j,
                -0.438616238538528 + 0.898674465693954j,
                0.990902635427780 - 0.134580708507126j,
                -0.990902635427780 + 0.134580708507126j,
                0.134580708507126 + 0.990902635427780j,
                -0.134580708507126 - 0.990902635427780j,
                0.795836904608884 + 0.605511041404325j,
                -0.795836904608884 - 0.605511041404325j,
                0.605511041404326 - 0.795836904608883j,
                -0.605511041404326 + 0.795836904608883j,
                0.966976471044852 + 0.254865659604515j,
                -0.966976471044852 - 0.254865659604515j,
                0.254865659604515 - 0.966976471044852j,
                -0.254865659604515 + 0.966976471044852j,
                0.863972856121587 - 0.503538383725718j,
                -0.863972856121587 + 0.503538383725718j,
                0.503538383725718 + 0.863972856121587j,
                -0.503538383725718 - 0.863972856121587j,
                0.999924701839145 - 0.0122715382857199j,
                -0.999924701839145 + 0.0122715382857199j,
                0.0122715382857199 + 0.999924701839145j,
                -0.0122715382857199 - 0.999924701839145j,
                0.715730825283819 + 0.698376249408973j,
                -0.715730825283819 - 0.698376249408973j,
                0.698376249408973 - 0.715730825283819j,
                -0.698376249408973 + 0.715730825283819j,
                0.928506080473216 + 0.371317193951837j,
                -0.928506080473216 - 0.371317193951837j,
                0.371317193951837 - 0.928506080473216j,
                -0.371317193951837 + 0.928506080473216j,
                0.919113851690058 - 0.393992040061048j,
                -0.919113851690058 + 0.393992040061048j,
                0.393992040061048 + 0.919113851690058j,
                -0.393992040061048 - 0.919113851690058j,
                0.983105487431216 + 0.183039887955141j,
                -0.983105487431216 - 0.183039887955141j,
                0.183039887955141 - 0.983105487431216j,
                -0.183039887955141 + 0.983105487431216j,
                0.824589302785025 - 0.565731810783613j,
                -0.824589302785025 + 0.565731810783613j,
                0.565731810783613 + 0.824589302785025j,
                -0.565731810783613 - 0.824589302785025j,
                0.978317370719628 - 0.207111376192219j,
                -0.978317370719628 + 0.207111376192219j,
                0.207111376192219 + 0.978317370719628j,
                -0.207111376192219 - 0.978317370719628j,
                0.838224705554838 + 0.545324988422046j,
                -0.838224705554838 - 0.545324988422046j,
                0.545324988422046 - 0.838224705554838j,
                -0.545324988422046 + 0.838224705554838j,
                0.996312612182778 + 0.0857973123444399j,
                -0.996312612182778 - 0.0857973123444399j,
                0.0857973123444399 - 0.996312612182778j,
                -0.0857973123444399 + 0.996312612182778j,
                0.765167265622459 - 0.643831542889791j,
                -0.765167265622459 + 0.643831542889791j,
                0.643831542889791 + 0.765167265622459j,
                -0.643831542889791 - 0.765167265622459j,
                0.953306040354194 - 0.302005949319228j,
                -0.953306040354194 + 0.302005949319228j,
                0.302005949319228 + 0.953306040354194j,
                -0.302005949319228 - 0.953306040354194j,
                0.887639620402854 + 0.460538710958240j,
                -0.887639620402854 - 0.460538710958240j,
                0.460538710958240 - 0.887639620402854j,
                -0.460538710958240 + 0.887639620402854j,
                0.993906970002356 - 0.110222207293883j,
                -0.993906970002356 + 0.110222207293883j,
                0.110222207293883 + 0.993906970002356j,
                -0.110222207293883 - 0.993906970002356j,
                0.780737228572094 + 0.624859488142386j,
                -0.780737228572094 - 0.624859488142386j,
                0.624859488142386 - 0.780737228572094j,
                -0.624859488142386 + 0.780737228572094j,
                0.960430519415566 + 0.278519689385053j,
                -0.960430519415566 - 0.278519689385053j,
                0.278519689385053 - 0.960430519415566j,
                -0.278519689385053 + 0.960430519415566j,
                0.876070094195407 - 0.482183772079123j,
                -0.876070094195407 + 0.482183772079123j,
                0.482183772079123 + 0.876070094195407j,
                -0.482183772079123 - 0.876070094195407j,
                0.999322384588350 + 0.0368072229413588j,
                -0.999322384588350 - 0.0368072229413588j,
                0.0368072229413588 - 0.999322384588350j,
                -0.0368072229413588 + 0.999322384588350j,
                0.732654271672413 - 0.680600997795453j,
                -0.732654271672413 + 0.680600997795453j,
                0.680600997795453 + 0.732654271672413j,
                -0.680600997795453 - 0.732654271672413j,
                0.937339011912575 - 0.348418680249435j,
                -0.937339011912575 + 0.348418680249435j,
                0.348418680249435 + 0.937339011912575j,
                -0.348418680249435 - 0.937339011912575j,
                0.909167983090522 + 0.416429560097637j,
                -0.909167983090522 - 0.416429560097637j,
                0.416429560097637 - 0.909167983090522j,
                -0.416429560097637 + 0.909167983090522j,
                0.987301418157858 - 0.158858143333861j,
                -0.987301418157858 + 0.158858143333861j,
                0.158858143333861 + 0.987301418157858j,
                -0.158858143333861 - 0.987301418157858j,
                0.810457198252595 + 0.585797857456439j,
                -0.810457198252595 - 0.585797857456439j,
                0.585797857456439 - 0.810457198252595j,
                -0.585797857456439 + 0.810457198252595j,
                0.972939952205560 + 0.231058108280671j,
                -0.972939952205560 - 0.231058108280671j,
                0.231058108280671 - 0.972939952205560j,
                -0.231058108280671 + 0.972939952205560j,
                0.851355193105265 - 0.524589682678469j,
                -0.851355193105265 + 0.524589682678469j,
                0.524589682678469 + 0.851355193105265j,
                -0.524589682678469 - 0.851355193105265j,
                0.998118112900149 - 0.0613207363022086j,
                -0.998118112900149 + 0.0613207363022086j,
                0.0613207363022086 + 0.998118112900149j,
                -0.0613207363022086 - 0.998118112900149j,
                0.749136394523459 + 0.662415777590172j,
                -0.749136394523459 - 0.662415777590172j,
                0.662415777590172 - 0.749136394523459j,
                -0.662415777590172 + 0.749136394523459j,
                0.945607325380521 + 0.325310292162263j,
                -0.945607325380521 - 0.325310292162263j,
                0.325310292162263 - 0.945607325380521j,
                -0.325310292162263 + 0.945607325380521j,
                0.898674465693954 - 0.438616238538528j,
                -0.898674465693954 + 0.438616238538528j,
                0.438616238538528 + 0.898674465693954j,
                -0.438616238538528 - 0.898674465693954j,
                0.990902635427780 + 0.134580708507126j,
                -0.990902635427780 - 0.134580708507126j,
                0.134580708507126 - 0.990902635427780j,
                -0.134580708507126 + 0.990902635427780j,
                0.795836904608884 - 0.605511041404325j,
                -0.795836904608884 + 0.605511041404325j,
                0.605511041404326 + 0.795836904608883j,
                -0.605511041404326 - 0.795836904608883j,
                0.966976471044852 - 0.254865659604515j,
                -0.966976471044852 + 0.254865659604515j,
                0.254865659604515 + 0.966976471044852j,
                -0.254865659604515 - 0.966976471044852j,
                0.863972856121587 + 0.503538383725718j,
                -0.863972856121587 - 0.503538383725718j,
                0.503538383725718 - 0.863972856121587j,
                -0.503538383725718 + 0.863972856121587j]


""" The roots of phi_1024 = x^512 + 1
The second half of the roots are is the conjugates of the first half.
The root at index (2 * i + 1) is the negation of the root at index (2 * i).
"""
phi1024_roots = [0.999981175282601 + 0.00613588464915448j,
                 -0.999981175282601 - 0.00613588464915448j,
                 0.00613588464915448 - 0.999981175282601j,
                 -0.00613588464915448 + 0.999981175282601j,
                 0.711432195745216 - 0.702754744457225j,
                 -0.711432195745216 + 0.702754744457225j,
                 0.702754744457225 + 0.711432195745216j,
                 -0.702754744457225 - 0.711432195745216j,
                 0.926210242138311 - 0.377007410216418j,
                 -0.926210242138311 + 0.377007410216418j,
                 0.377007410216418 + 0.926210242138311j,
                 -0.377007410216418 - 0.926210242138311j,
                 0.921514039342042 + 0.388345046698826j,
                 -0.921514039342042 - 0.388345046698826j,
                 0.388345046698826 - 0.921514039342042j,
                 -0.388345046698826 + 0.921514039342042j,
                 0.981963869109555 - 0.189068664149806j,
                 -0.981963869109555 + 0.189068664149806j,
                 0.189068664149806 + 0.981963869109555j,
                 -0.189068664149806 - 0.981963869109555j,
                 0.828045045257756 + 0.560661576197336j,
                 -0.828045045257756 - 0.560661576197336j,
                 0.560661576197336 - 0.828045045257756j,
                 -0.560661576197336 + 0.828045045257756j,
                 0.979569765685441 + 0.201104634842092j,
                 -0.979569765685441 - 0.201104634842092j,
                 0.201104634842092 - 0.979569765685441j,
                 -0.201104634842092 + 0.979569765685441j,
                 0.834862874986380 - 0.550457972936605j,
                 -0.834862874986380 + 0.550457972936605j,
                 0.550457972936605 + 0.834862874986380j,
                 -0.550457972936605 - 0.834862874986380j,
                 0.995767414467660 - 0.0919089564971327j,
                 -0.995767414467660 + 0.0919089564971327j,
                 0.0919089564971327 + 0.995767414467660j,
                 -0.0919089564971327 - 0.995767414467660j,
                 0.769103337645580 + 0.639124444863776j,
                 -0.769103337645580 - 0.639124444863776j,
                 0.639124444863776 - 0.769103337645580j,
                 -0.639124444863776 + 0.769103337645580j,
                 0.955141168305771 + 0.296150888243624j,
                 -0.955141168305771 - 0.296150888243624j,
                 0.296150888243624 - 0.955141168305771j,
                 -0.296150888243624 + 0.955141168305771j,
                 0.884797098430938 - 0.465976495767966j,
                 -0.884797098430938 + 0.465976495767966j,
                 0.465976495767966 + 0.884797098430938j,
                 -0.465976495767966 - 0.884797098430938j,
                 0.994564570734255 + 0.104121633872055j,
                 -0.994564570734255 - 0.104121633872055j,
                 0.104121633872055 - 0.994564570734255j,
                 -0.104121633872055 + 0.994564570734255j,
                 0.776888465673232 - 0.629638238914927j,
                 -0.776888465673232 + 0.629638238914927j,
                 0.629638238914927 + 0.776888465673233j,
                 -0.629638238914927 - 0.776888465673233j,
                 0.958703474895871 - 0.284407537211272j,
                 -0.958703474895871 + 0.284407537211272j,
                 0.284407537211272 + 0.958703474895871j,
                 -0.284407537211272 - 0.958703474895871j,
                 0.879012226428633 + 0.476799230063322j,
                 -0.879012226428633 - 0.476799230063322j,
                 0.476799230063322 - 0.879012226428634j,
                 -0.476799230063322 + 0.879012226428634j,
                 0.999077727752645 - 0.0429382569349408j,
                 -0.999077727752645 + 0.0429382569349408j,
                 0.0429382569349408 + 0.999077727752645j,
                 -0.0429382569349408 - 0.999077727752645j,
                 0.736816568877370 + 0.676092703575316j,
                 -0.736816568877370 - 0.676092703575316j,
                 0.676092703575316 - 0.736816568877370j,
                 -0.676092703575316 + 0.736816568877370j,
                 0.939459223602190 + 0.342660717311994j,
                 -0.939459223602190 - 0.342660717311994j,
                 0.342660717311994 - 0.939459223602190j,
                 -0.342660717311994 + 0.939459223602190j,
                 0.906595704514915 - 0.422000270799800j,
                 -0.906595704514915 + 0.422000270799800j,
                 0.422000270799800 + 0.906595704514915j,
                 -0.422000270799800 - 0.906595704514915j,
                 0.988257567730749 + 0.152797185258443j,
                 -0.988257567730749 - 0.152797185258443j,
                 0.152797185258443 - 0.988257567730749j,
                 -0.152797185258443 + 0.988257567730749j,
                 0.806847553543799 - 0.590759701858874j,
                 -0.806847553543799 + 0.590759701858874j,
                 0.590759701858874 + 0.806847553543799j,
                 -0.590759701858874 - 0.806847553543799j,
                 0.971503890986252 - 0.237023605994367j,
                 -0.971503890986252 + 0.237023605994367j,
                 0.237023605994367 + 0.971503890986252j,
                 -0.237023605994367 - 0.971503890986252j,
                 0.854557988365401 + 0.519355990165590j,
                 -0.854557988365401 - 0.519355990165590j,
                 0.519355990165590 - 0.854557988365401j,
                 -0.519355990165590 + 0.854557988365401j,
                 0.998475580573295 + 0.0551952443496899j,
                 -0.998475580573295 - 0.0551952443496899j,
                 0.0551952443496899 - 0.998475580573295j,
                 -0.0551952443496899 + 0.998475580573295j,
                 0.745057785441466 - 0.666999922303638j,
                 -0.745057785441466 + 0.666999922303638j,
                 0.666999922303637 + 0.745057785441466j,
                 -0.666999922303637 - 0.745057785441466j,
                 0.943593458161960 - 0.331106305759876j,
                 -0.943593458161960 + 0.331106305759876j,
                 0.331106305759876 + 0.943593458161960j,
                 -0.331106305759876 - 0.943593458161960j,
                 0.901348847046022 + 0.433093818853152j,
                 -0.901348847046022 - 0.433093818853152j,
                 0.433093818853152 - 0.901348847046022j,
                 -0.433093818853152 + 0.901348847046022j,
                 0.990058210262297 - 0.140658239332849j,
                 -0.990058210262297 + 0.140658239332849j,
                 0.140658239332849 + 0.990058210262297j,
                 -0.140658239332849 - 0.990058210262297j,
                 0.799537269107905 + 0.600616479383869j,
                 -0.799537269107905 - 0.600616479383869j,
                 0.600616479383869 - 0.799537269107905j,
                 -0.600616479383869 + 0.799537269107905j,
                 0.968522094274417 + 0.248927605745720j,
                 -0.968522094274417 - 0.248927605745720j,
                 0.248927605745720 - 0.968522094274417j,
                 -0.248927605745720 + 0.968522094274417j,
                 0.860866938637767 - 0.508830142543107j,
                 -0.860866938637767 + 0.508830142543107j,
                 0.508830142543107 + 0.860866938637767j,
                 -0.508830142543107 - 0.860866938637767j,
                 0.999830581795823 - 0.0184067299058048j,
                 -0.999830581795823 + 0.0184067299058048j,
                 0.0184067299058048 + 0.999830581795823j,
                 -0.0184067299058048 - 0.999830581795823j,
                 0.720002507961382 + 0.693971460889654j,
                 -0.720002507961382 - 0.693971460889654j,
                 0.693971460889654 - 0.720002507961382j,
                 -0.693971460889654 + 0.720002507961382j,
                 0.930766961078984 + 0.365612997804774j,
                 -0.930766961078984 - 0.365612997804774j,
                 0.365612997804774 - 0.930766961078984j,
                 -0.365612997804774 + 0.930766961078984j,
                 0.916679059921043 - 0.399624199845647j,
                 -0.916679059921043 + 0.399624199845647j,
                 0.399624199845647 + 0.916679059921043j,
                 -0.399624199845647 - 0.916679059921043j,
                 0.984210092386929 + 0.177004220412149j,
                 -0.984210092386929 - 0.177004220412149j,
                 0.177004220412149 - 0.984210092386929j,
                 -0.177004220412149 + 0.984210092386929j,
                 0.821102514991105 - 0.570780745886967j,
                 -0.821102514991105 + 0.570780745886967j,
                 0.570780745886967 + 0.821102514991105j,
                 -0.570780745886967 - 0.821102514991105j,
                 0.977028142657754 - 0.213110319916091j,
                 -0.977028142657754 + 0.213110319916091j,
                 0.213110319916091 + 0.977028142657754j,
                 -0.213110319916091 - 0.977028142657754j,
                 0.841554977436898 + 0.540171472729893j,
                 -0.841554977436898 - 0.540171472729893j,
                 0.540171472729893 - 0.841554977436898j,
                 -0.540171472729893 + 0.841554977436898j,
                 0.996820299291166 + 0.0796824379714301j,
                 -0.996820299291166 - 0.0796824379714301j,
                 0.0796824379714301 - 0.996820299291166j,
                 -0.0796824379714301 + 0.996820299291166j,
                 0.761202385484262 - 0.648514401022113j,
                 -0.761202385484262 + 0.648514401022113j,
                 0.648514401022112 + 0.761202385484262j,
                 -0.648514401022112 - 0.761202385484262j,
                 0.951435020969008 - 0.307849640041535j,
                 -0.951435020969008 + 0.307849640041535j,
                 0.307849640041535 + 0.951435020969008j,
                 -0.307849640041535 - 0.951435020969008j,
                 0.890448723244758 + 0.455083587126344j,
                 -0.890448723244758 - 0.455083587126344j,
                 0.455083587126344 - 0.890448723244758j,
                 -0.455083587126344 + 0.890448723244758j,
                 0.993211949234795 - 0.116318630911905j,
                 -0.993211949234795 + 0.116318630911905j,
                 0.116318630911905 + 0.993211949234795j,
                 -0.116318630911905 - 0.993211949234795j,
                 0.784556597155575 + 0.620057211763289j,
                 -0.784556597155575 - 0.620057211763289j,
                 0.620057211763289 - 0.784556597155575j,
                 -0.620057211763289 + 0.784556597155575j,
                 0.962121404269042 + 0.272621355449949j,
                 -0.962121404269042 - 0.272621355449949j,
                 0.272621355449949 - 0.962121404269042j,
                 -0.272621355449949 + 0.962121404269042j,
                 0.873094978418290 - 0.487550160148436j,
                 -0.873094978418290 + 0.487550160148436j,
                 0.487550160148436 + 0.873094978418290j,
                 -0.487550160148436 - 0.873094978418290j,
                 0.999529417501093 + 0.0306748031766366j,
                 -0.999529417501093 - 0.0306748031766366j,
                 0.0306748031766366 - 0.999529417501093j,
                 -0.0306748031766366 + 0.999529417501093j,
                 0.728464390448225 - 0.685083667772700j,
                 -0.728464390448225 + 0.685083667772700j,
                 0.685083667772700 + 0.728464390448225j,
                 -0.685083667772700 - 0.728464390448225j,
                 0.935183509938948 - 0.354163525420490j,
                 -0.935183509938948 + 0.354163525420490j,
                 0.354163525420490 + 0.935183509938948j,
                 -0.354163525420490 - 0.935183509938948j,
                 0.911706032005430 + 0.410843171057904j,
                 -0.911706032005430 - 0.410843171057904j,
                 0.410843171057904 - 0.911706032005430j,
                 -0.410843171057904 + 0.911706032005430j,
                 0.986308097244599 - 0.164913120489970j,
                 -0.986308097244599 + 0.164913120489970j,
                 0.164913120489970 + 0.986308097244599j,
                 -0.164913120489970 - 0.986308097244599j,
                 0.814036329705948 + 0.580813958095765j,
                 -0.814036329705948 - 0.580813958095765j,
                 0.580813958095765 - 0.814036329705948j,
                 -0.580813958095765 + 0.814036329705948j,
                 0.974339382785576 + 0.225083911359793j,
                 -0.974339382785576 - 0.225083911359793j,
                 0.225083911359793 - 0.974339382785576j,
                 -0.225083911359793 + 0.974339382785576j,
                 0.848120344803297 - 0.529803624686295j,
                 -0.848120344803297 + 0.529803624686295j,
                 0.529803624686295 + 0.848120344803297j,
                 -0.529803624686295 - 0.848120344803297j,
                 0.997723066644192 - 0.0674439195636641j,
                 -0.997723066644192 + 0.0674439195636641j,
                 0.0674439195636641 + 0.997723066644192j,
                 -0.0674439195636641 - 0.997723066644192j,
                 0.753186799043613 + 0.657806693297079j,
                 -0.753186799043613 - 0.657806693297079j,
                 0.657806693297079 - 0.753186799043613j,
                 -0.657806693297079 + 0.753186799043613j,
                 0.947585591017741 + 0.319502030816016j,
                 -0.947585591017741 - 0.319502030816016j,
                 0.319502030816016 - 0.947585591017741j,
                 -0.319502030816016 + 0.947585591017741j,
                 0.895966249756185 - 0.444122144570429j,
                 -0.895966249756185 + 0.444122144570429j,
                 0.444122144570429 + 0.895966249756185j,
                 -0.444122144570429 - 0.895966249756185j,
                 0.991709753669100 + 0.128498110793793j,
                 -0.991709753669100 - 0.128498110793793j,
                 0.128498110793793 - 0.991709753669100j,
                 -0.128498110793793 + 0.991709753669100j,
                 0.792106577300212 - 0.610382806276309j,
                 -0.792106577300212 + 0.610382806276309j,
                 0.610382806276309 + 0.792106577300212j,
                 -0.610382806276309 - 0.792106577300212j,
                 0.965394441697689 - 0.260794117915276j,
                 -0.965394441697689 + 0.260794117915276j,
                 0.260794117915276 + 0.965394441697689j,
                 -0.260794117915276 - 0.965394441697689j,
                 0.867046245515693 + 0.498227666972782j,
                 -0.867046245515693 - 0.498227666972782j,
                 0.498227666972782 - 0.867046245515693j,
                 -0.498227666972782 + 0.867046245515693j,
                 0.999981175282601 - 0.00613588464915448j,
                 -0.999981175282601 + 0.00613588464915448j,
                 0.00613588464915448 + 0.999981175282601j,
                 -0.00613588464915448 - 0.999981175282601j,
                 0.711432195745216 + 0.702754744457225j,
                 -0.711432195745216 - 0.702754744457225j,
                 0.702754744457225 - 0.711432195745216j,
                 -0.702754744457225 + 0.711432195745216j,
                 0.926210242138311 + 0.377007410216418j,
                 -0.926210242138311 - 0.377007410216418j,
                 0.377007410216418 - 0.926210242138311j,
                 -0.377007410216418 + 0.926210242138311j,
                 0.921514039342042 - 0.388345046698826j,
                 -0.921514039342042 + 0.388345046698826j,
                 0.388345046698826 + 0.921514039342042j,
                 -0.388345046698826 - 0.921514039342042j,
                 0.981963869109555 + 0.189068664149806j,
                 -0.981963869109555 - 0.189068664149806j,
                 0.189068664149806 - 0.981963869109555j,
                 -0.189068664149806 + 0.981963869109555j,
                 0.828045045257756 - 0.560661576197336j,
                 -0.828045045257756 + 0.560661576197336j,
                 0.560661576197336 + 0.828045045257756j,
                 -0.560661576197336 - 0.828045045257756j,
                 0.979569765685441 - 0.201104634842092j,
                 -0.979569765685441 + 0.201104634842092j,
                 0.201104634842092 + 0.979569765685441j,
                 -0.201104634842092 - 0.979569765685441j,
                 0.834862874986380 + 0.550457972936605j,
                 -0.834862874986380 - 0.550457972936605j,
                 0.550457972936605 - 0.834862874986380j,
                 -0.550457972936605 + 0.834862874986380j,
                 0.995767414467660 + 0.0919089564971327j,
                 -0.995767414467660 - 0.0919089564971327j,
                 0.0919089564971327 - 0.995767414467660j,
                 -0.0919089564971327 + 0.995767414467660j,
                 0.769103337645580 - 0.639124444863776j,
                 -0.769103337645580 + 0.639124444863776j,
                 0.639124444863776 + 0.769103337645580j,
                 -0.639124444863776 - 0.769103337645580j,
                 0.955141168305771 - 0.296150888243624j,
                 -0.955141168305771 + 0.296150888243624j,
                 0.296150888243624 + 0.955141168305771j,
                 -0.296150888243624 - 0.955141168305771j,
                 0.884797098430938 + 0.465976495767966j,
                 -0.884797098430938 - 0.465976495767966j,
                 0.465976495767966 - 0.884797098430938j,
                 -0.465976495767966 + 0.884797098430938j,
                 0.994564570734255 - 0.104121633872055j,
                 -0.994564570734255 + 0.104121633872055j,
                 0.104121633872055 + 0.994564570734255j,
                 -0.104121633872055 - 0.994564570734255j,
                 0.776888465673232 + 0.629638238914927j,
                 -0.776888465673232 - 0.629638238914927j,
                 0.629638238914927 - 0.776888465673233j,
                 -0.629638238914927 + 0.776888465673233j,
                 0.958703474895871 + 0.284407537211272j,
                 -0.958703474895871 - 0.284407537211272j,
                 0.284407537211272 - 0.958703474895871j,
                 -0.284407537211272 + 0.958703474895871j,
                 0.879012226428633 - 0.476799230063322j,
                 -0.879012226428633 + 0.476799230063322j,
                 0.476799230063322 + 0.879012226428634j,
                 -0.476799230063322 - 0.879012226428634j,
                 0.999077727752645 + 0.0429382569349408j,
                 -0.999077727752645 - 0.0429382569349408j,
                 0.0429382569349408 - 0.999077727752645j,
                 -0.0429382569349408 + 0.999077727752645j,
                 0.736816568877370 - 0.676092703575316j,
                 -0.736816568877370 + 0.676092703575316j,
                 0.676092703575316 + 0.736816568877370j,
                 -0.676092703575316 - 0.736816568877370j,
                 0.939459223602190 - 0.342660717311994j,
                 -0.939459223602190 + 0.342660717311994j,
                 0.342660717311994 + 0.939459223602190j,
                 -0.342660717311994 - 0.939459223602190j,
                 0.906595704514915 + 0.422000270799800j,
                 -0.906595704514915 - 0.422000270799800j,
                 0.422000270799800 - 0.906595704514915j,
                 -0.422000270799800 + 0.906595704514915j,
                 0.988257567730749 - 0.152797185258443j,
                 -0.988257567730749 + 0.152797185258443j,
                 0.152797185258443 + 0.988257567730749j,
                 -0.152797185258443 - 0.988257567730749j,
                 0.806847553543799 + 0.590759701858874j,
                 -0.806847553543799 - 0.590759701858874j,
                 0.590759701858874 - 0.806847553543799j,
                 -0.590759701858874 + 0.806847553543799j,
                 0.971503890986252 + 0.237023605994367j,
                 -0.971503890986252 - 0.237023605994367j,
                 0.237023605994367 - 0.971503890986252j,
                 -0.237023605994367 + 0.971503890986252j,
                 0.854557988365401 - 0.519355990165590j,
                 -0.854557988365401 + 0.519355990165590j,
                 0.519355990165590 + 0.854557988365401j,
                 -0.519355990165590 - 0.854557988365401j,
                 0.998475580573295 - 0.0551952443496899j,
                 -0.998475580573295 + 0.0551952443496899j,
                 0.0551952443496899 + 0.998475580573295j,
                 -0.0551952443496899 - 0.998475580573295j,
                 0.745057785441466 + 0.666999922303638j,
                 -0.745057785441466 - 0.666999922303638j,
                 0.666999922303637 - 0.745057785441466j,
                 -0.666999922303637 + 0.745057785441466j,
                 0.943593458161960 + 0.331106305759876j,
                 -0.943593458161960 - 0.331106305759876j,
                 0.331106305759876 - 0.943593458161960j,
                 -0.331106305759876 + 0.943593458161960j,
                 0.901348847046022 - 0.433093818853152j,
                 -0.901348847046022 + 0.433093818853152j,
                 0.433093818853152 + 0.901348847046022j,
                 -0.433093818853152 - 0.901348847046022j,
                 0.990058210262297 + 0.140658239332849j,
                 -0.990058210262297 - 0.140658239332849j,
                 0.140658239332849 - 0.990058210262297j,
                 -0.140658239332849 + 0.990058210262297j,
                 0.799537269107905 - 0.600616479383869j,
                 -0.799537269107905 + 0.600616479383869j,
                 0.600616479383869 + 0.799537269107905j,
                 -0.600616479383869 - 0.799537269107905j,
                 0.968522094274417 - 0.248927605745720j,
                 -0.968522094274417 + 0.248927605745720j,
                 0.248927605745720 + 0.968522094274417j,
                 -0.248927605745720 - 0.968522094274417j,
                 0.860866938637767 + 0.508830142543107j,
                 -0.860866938637767 - 0.508830142543107j,
                 0.508830142543107 - 0.860866938637767j,
                 -0.508830142543107 + 0.860866938637767j,
                 0.999830581795823 + 0.0184067299058048j,
                 -0.999830581795823 - 0.0184067299058048j,
                 0.0184067299058048 - 0.999830581795823j,
                 -0.0184067299058048 + 0.999830581795823j,
                 0.720002507961382 - 0.693971460889654j,
                 -0.720002507961382 + 0.693971460889654j,
                 0.693971460889654 + 0.720002507961382j,
                 -0.693971460889654 - 0.720002507961382j,
                 0.930766961078984 - 0.365612997804774j,
                 -0.930766961078984 + 0.365612997804774j,
                 0.365612997804774 + 0.930766961078984j,
                 -0.365612997804774 - 0.930766961078984j,
                 0.916679059921043 + 0.399624199845647j,
                 -0.916679059921043 - 0.399624199845647j,
                 0.399624199845647 - 0.916679059921043j,
                 -0.399624199845647 + 0.916679059921043j,
                 0.984210092386929 - 0.177004220412149j,
                 -0.984210092386929 + 0.177004220412149j,
                 0.177004220412149 + 0.984210092386929j,
                 -0.177004220412149 - 0.984210092386929j,
                 0.821102514991105 + 0.570780745886967j,
                 -0.821102514991105 - 0.570780745886967j,
                 0.570780745886967 - 0.821102514991105j,
                 -0.570780745886967 + 0.821102514991105j,
                 0.977028142657754 + 0.213110319916091j,
                 -0.977028142657754 - 0.213110319916091j,
                 0.213110319916091 - 0.977028142657754j,
                 -0.213110319916091 + 0.977028142657754j,
                 0.841554977436898 - 0.540171472729893j,
                 -0.841554977436898 + 0.540171472729893j,
                 0.540171472729893 + 0.841554977436898j,
                 -0.540171472729893 - 0.841554977436898j,
                 0.996820299291166 - 0.0796824379714301j,
                 -0.996820299291166 + 0.0796824379714301j,
                 0.0796824379714301 + 0.996820299291166j,
                 -0.0796824379714301 - 0.996820299291166j,
                 0.761202385484262 + 0.648514401022113j,
                 -0.761202385484262 - 0.648514401022113j,
                 0.648514401022112 - 0.761202385484262j,
                 -0.648514401022112 + 0.761202385484262j,
                 0.951435020969008 + 0.307849640041535j,
                 -0.951435020969008 - 0.307849640041535j,
                 0.307849640041535 - 0.951435020969008j,
                 -0.307849640041535 + 0.951435020969008j,
                 0.890448723244758 - 0.455083587126344j,
                 -0.890448723244758 + 0.455083587126344j,
                 0.455083587126344 + 0.890448723244758j,
                 -0.455083587126344 - 0.890448723244758j,
                 0.993211949234795 + 0.116318630911905j,
                 -0.993211949234795 - 0.116318630911905j,
                 0.116318630911905 - 0.993211949234795j,
                 -0.116318630911905 + 0.993211949234795j,
                 0.784556597155575 - 0.620057211763289j,
                 -0.784556597155575 + 0.620057211763289j,
                 0.620057211763289 + 0.784556597155575j,
                 -0.620057211763289 - 0.784556597155575j,
                 0.962121404269042 - 0.272621355449949j,
                 -0.962121404269042 + 0.272621355449949j,
                 0.272621355449949 + 0.962121404269042j,
                 -0.272621355449949 - 0.962121404269042j,
                 0.873094978418290 + 0.487550160148436j,
                 -0.873094978418290 - 0.487550160148436j,
                 0.487550160148436 - 0.873094978418290j,
                 -0.487550160148436 + 0.873094978418290j,
                 0.999529417501093 - 0.0306748031766366j,
                 -0.999529417501093 + 0.0306748031766366j,
                 0.0306748031766366 + 0.999529417501093j,
                 -0.0306748031766366 - 0.999529417501093j,
                 0.728464390448225 + 0.685083667772700j,
                 -0.728464390448225 - 0.685083667772700j,
                 0.685083667772700 - 0.728464390448225j,
                 -0.685083667772700 + 0.728464390448225j,
                 0.935183509938948 + 0.354163525420490j,
                 -0.935183509938948 - 0.354163525420490j,
                 0.354163525420490 - 0.935183509938948j,
                 -0.354163525420490 + 0.935183509938948j,
                 0.911706032005430 - 0.410843171057904j,
                 -0.911706032005430 + 0.410843171057904j,
                 0.410843171057904 + 0.911706032005430j,
                 -0.410843171057904 - 0.911706032005430j,
                 0.986308097244599 + 0.164913120489970j,
                 -0.986308097244599 - 0.164913120489970j,
                 0.164913120489970 - 0.986308097244599j,
                 -0.164913120489970 + 0.986308097244599j,
                 0.814036329705948 - 0.580813958095765j,
                 -0.814036329705948 + 0.580813958095765j,
                 0.580813958095765 + 0.814036329705948j,
                 -0.580813958095765 - 0.814036329705948j,
                 0.974339382785576 - 0.225083911359793j,
                 -0.974339382785576 + 0.225083911359793j,
                 0.225083911359793 + 0.974339382785576j,
                 -0.225083911359793 - 0.974339382785576j,
                 0.848120344803297 + 0.529803624686295j,
                 -0.848120344803297 - 0.529803624686295j,
                 0.529803624686295 - 0.848120344803297j,
                 -0.529803624686295 + 0.848120344803297j,
                 0.997723066644192 + 0.0674439195636641j,
                 -0.997723066644192 - 0.0674439195636641j,
                 0.0674439195636641 - 0.997723066644192j,
                 -0.0674439195636641 + 0.997723066644192j,
                 0.753186799043613 - 0.657806693297079j,
                 -0.753186799043613 + 0.657806693297079j,
                 0.657806693297079 + 0.753186799043613j,
                 -0.657806693297079 - 0.753186799043613j,
                 0.947585591017741 - 0.319502030816016j,
                 -0.947585591017741 + 0.319502030816016j,
                 0.319502030816016 + 0.947585591017741j,
                 -0.319502030816016 - 0.947585591017741j,
                 0.895966249756185 + 0.444122144570429j,
                 -0.895966249756185 - 0.444122144570429j,
                 0.444122144570429 - 0.895966249756185j,
                 -0.444122144570429 + 0.895966249756185j,
                 0.991709753669100 - 0.128498110793793j,
                 -0.991709753669100 + 0.128498110793793j,
                 0.128498110793793 + 0.991709753669100j,
                 -0.128498110793793 - 0.991709753669100j,
                 0.792106577300212 + 0.610382806276309j,
                 -0.792106577300212 - 0.610382806276309j,
                 0.610382806276309 - 0.792106577300212j,
                 -0.610382806276309 + 0.792106577300212j,
                 0.965394441697689 + 0.260794117915276j,
                 -0.965394441697689 - 0.260794117915276j,
                 0.260794117915276 - 0.965394441697689j,
                 -0.260794117915276 + 0.965394441697689j,
                 0.867046245515693 - 0.498227666972782j,
                 -0.867046245515693 + 0.498227666972782j,
                 0.498227666972782 + 0.867046245515693j,
                 -0.498227666972782 - 0.867046245515693j]


""" The roots of phi_2048 = x^1024 + 1
The second half of the roots are is the conjugates of the first half.
The root at index (2 * i + 1) is the negation of the root at index (2 * i).
"""
phi2048_roots = [0.999995293809576 + 0.00306795676296598j,
                 -0.999995293809576 - 0.00306795676296598j,
                 0.00306795676296598 - 0.999995293809576j,
                 -0.00306795676296598 + 0.999995293809576j,
                 0.709272826438866 - 0.704934080375905j,
                 -0.709272826438866 + 0.704934080375905j,
                 0.704934080375905 + 0.709272826438866j,
                 -0.704934080375905 - 0.709272826438866j,
                 0.925049240782678 - 0.379847208924051j,
                 -0.925049240782678 + 0.379847208924051j,
                 0.379847208924051 + 0.925049240782678j,
                 -0.379847208924051 - 0.925049240782678j,
                 0.922701128333879 + 0.385516053843919j,
                 -0.922701128333879 - 0.385516053843919j,
                 0.385516053843919 - 0.922701128333879j,
                 -0.385516053843919 + 0.922701128333879j,
                 0.981379193313755 - 0.192080397049892j,
                 -0.981379193313755 + 0.192080397049892j,
                 0.192080397049892 + 0.981379193313755j,
                 -0.192080397049892 - 0.981379193313755j,
                 0.829761233794523 + 0.558118531220556j,
                 -0.829761233794523 - 0.558118531220556j,
                 0.558118531220556 - 0.829761233794523j,
                 -0.558118531220556 + 0.829761233794523j,
                 0.980182135968117 + 0.198098410717954j,
                 -0.980182135968117 - 0.198098410717954j,
                 0.198098410717954 - 0.980182135968117j,
                 -0.198098410717954 + 0.980182135968117j,
                 0.833170164701913 - 0.553016705580028j,
                 -0.833170164701913 + 0.553016705580028j,
                 0.553016705580028 + 0.833170164701913j,
                 -0.553016705580028 - 0.833170164701913j,
                 0.995480755491927 - 0.0949634953296390j,
                 -0.995480755491927 + 0.0949634953296390j,
                 0.0949634953296390 + 0.995480755491927j,
                 -0.0949634953296390 - 0.995480755491927j,
                 0.771060524261814 + 0.636761861236284j,
                 -0.771060524261814 - 0.636761861236284j,
                 0.636761861236284 - 0.771060524261814j,
                 -0.636761861236284 + 0.771060524261814j,
                 0.956045251349996 + 0.293219162694259j,
                 -0.956045251349996 - 0.293219162694259j,
                 0.293219162694259 - 0.956045251349996j,
                 -0.293219162694259 + 0.956045251349996j,
                 0.883363338665732 - 0.468688822035828j,
                 -0.883363338665732 + 0.468688822035828j,
                 0.468688822035828 + 0.883363338665732j,
                 -0.468688822035828 - 0.883363338665732j,
                 0.994879330794806 + 0.101069862754828j,
                 -0.994879330794806 - 0.101069862754828j,
                 0.101069862754828 - 0.994879330794806j,
                 -0.101069862754828 + 0.994879330794806j,
                 0.774953106594874 - 0.632018735939809j,
                 -0.774953106594874 + 0.632018735939809j,
                 0.632018735939809 + 0.774953106594874j,
                 -0.632018735939809 - 0.774953106594874j,
                 0.957826413027533 - 0.287347459544730j,
                 -0.957826413027533 + 0.287347459544730j,
                 0.287347459544730 + 0.957826413027533j,
                 -0.287347459544730 - 0.957826413027533j,
                 0.880470889052161 + 0.474100214650550j,
                 -0.880470889052161 - 0.474100214650550j,
                 0.474100214650550 - 0.880470889052161j,
                 -0.474100214650550 + 0.880470889052161j,
                 0.998941293186857 - 0.0460031821309146j,
                 -0.998941293186857 + 0.0460031821309146j,
                 0.0460031821309146 + 0.998941293186857j,
                 -0.0460031821309146 - 0.998941293186857j,
                 0.738887324460615 + 0.673829000378756j,
                 -0.738887324460615 - 0.673829000378756j,
                 0.673829000378756 - 0.738887324460615j,
                 -0.673829000378756 + 0.738887324460615j,
                 0.940506070593268 + 0.339776884406827j,
                 -0.940506070593268 - 0.339776884406827j,
                 0.339776884406827 - 0.940506070593268j,
                 -0.339776884406827 + 0.940506070593268j,
                 0.905296759318119 - 0.424779681209109j,
                 -0.905296759318119 + 0.424779681209109j,
                 0.424779681209109 + 0.905296759318119j,
                 -0.424779681209109 - 0.905296759318119j,
                 0.988721691960324 + 0.149764534677322j,
                 -0.988721691960324 - 0.149764534677322j,
                 0.149764534677322 - 0.988721691960324j,
                 -0.149764534677322 + 0.988721691960324j,
                 0.805031331142964 - 0.593232295039800j,
                 -0.805031331142964 + 0.593232295039800j,
                 0.593232295039800 + 0.805031331142964j,
                 -0.593232295039800 - 0.805031331142964j,
                 0.970772140728950 - 0.240003022448741j,
                 -0.970772140728950 + 0.240003022448741j,
                 0.240003022448741 + 0.970772140728950j,
                 -0.240003022448741 - 0.970772140728950j,
                 0.856147328375194 + 0.516731799017650j,
                 -0.856147328375194 - 0.516731799017650j,
                 0.516731799017650 - 0.856147328375194j,
                 -0.516731799017650 + 0.856147328375194j,
                 0.998640218180265 + 0.0521317046802833j,
                 -0.998640218180265 - 0.0521317046802833j,
                 0.0521317046802833 - 0.998640218180265j,
                 -0.0521317046802833 + 0.998640218180265j,
                 0.743007952135122 - 0.669282588346636j,
                 -0.743007952135122 + 0.669282588346636j,
                 0.669282588346636 + 0.743007952135122j,
                 -0.669282588346636 - 0.743007952135122j,
                 0.942573197601447 - 0.333999651442009j,
                 -0.942573197601447 + 0.333999651442009j,
                 0.333999651442009 + 0.942573197601447j,
                 -0.333999651442009 - 0.942573197601447j,
                 0.902673318237259 + 0.430326481340083j,
                 -0.902673318237259 - 0.430326481340083j,
                 0.430326481340083 - 0.902673318237259j,
                 -0.430326481340083 + 0.902673318237259j,
                 0.989622017463201 - 0.143695033150294j,
                 -0.989622017463201 + 0.143695033150294j,
                 0.143695033150294 + 0.989622017463201j,
                 -0.143695033150294 - 0.989622017463201j,
                 0.801376171723140 + 0.598160706996342j,
                 -0.801376171723140 - 0.598160706996342j,
                 0.598160706996342 - 0.801376171723140j,
                 -0.598160706996342 + 0.801376171723140j,
                 0.969281235356549 + 0.245955050335795j,
                 -0.969281235356549 - 0.245955050335795j,
                 0.245955050335795 - 0.969281235356549j,
                 -0.245955050335795 + 0.969281235356549j,
                 0.859301818357008 - 0.511468850437970j,
                 -0.859301818357008 + 0.511468850437970j,
                 0.511468850437970 + 0.859301818357008j,
                 -0.511468850437970 - 0.859301818357008j,
                 0.999769405351215 - 0.0214740802754695j,
                 -0.999769405351215 + 0.0214740802754695j,
                 0.0214740802754695 + 0.999769405351215j,
                 -0.0214740802754695 - 0.999769405351215j,
                 0.722128193929215 + 0.691759258364158j,
                 -0.722128193929215 - 0.691759258364158j,
                 0.691759258364158 - 0.722128193929215j,
                 -0.691759258364158 + 0.722128193929215j,
                 0.931884265581668 + 0.362755724367397j,
                 -0.931884265581668 - 0.362755724367397j,
                 0.362755724367397 - 0.931884265581668j,
                 -0.362755724367397 + 0.931884265581668j,
                 0.915448716088268 - 0.402434650859418j,
                 -0.915448716088268 + 0.402434650859418j,
                 0.402434650859418 + 0.915448716088268j,
                 -0.402434650859418 - 0.915448716088268j,
                 0.984748501801904 + 0.173983873387464j,
                 -0.984748501801904 - 0.173983873387464j,
                 0.173983873387464 - 0.984748501801904j,
                 -0.173983873387464 + 0.984748501801904j,
                 0.819347520076797 - 0.573297166698042j,
                 -0.819347520076797 + 0.573297166698042j,
                 0.573297166698042 + 0.819347520076797j,
                 -0.573297166698042 - 0.819347520076797j,
                 0.976369731330021 - 0.216106797076220j,
                 -0.976369731330021 + 0.216106797076220j,
                 0.216106797076220 + 0.976369731330021j,
                 -0.216106797076220 - 0.976369731330021j,
                 0.843208239641845 + 0.537587076295646j,
                 -0.843208239641845 - 0.537587076295646j,
                 0.537587076295646 - 0.843208239641845j,
                 -0.537587076295646 + 0.843208239641845j,
                 0.997060070339483 + 0.0766238613920315j,
                 -0.997060070339483 - 0.0766238613920315j,
                 0.0766238613920315 - 0.997060070339483j,
                 -0.0766238613920315 + 0.997060070339483j,
                 0.759209188978388 - 0.650846684996381j,
                 -0.759209188978388 + 0.650846684996381j,
                 0.650846684996381 + 0.759209188978388j,
                 -0.650846684996381 - 0.759209188978388j,
                 0.950486073949482 - 0.310767152749612j,
                 -0.950486073949482 + 0.310767152749612j,
                 0.310767152749612 + 0.950486073949482j,
                 -0.310767152749612 - 0.950486073949482j,
                 0.891840709392343 + 0.452349587233771j,
                 -0.891840709392343 - 0.452349587233771j,
                 0.452349587233771 - 0.891840709392343j,
                 -0.452349587233771 + 0.891840709392343j,
                 0.992850414459865 - 0.119365214810991j,
                 -0.992850414459865 + 0.119365214810991j,
                 0.119365214810991 + 0.992850414459865j,
                 -0.119365214810991 - 0.992850414459865j,
                 0.786455213599086 + 0.617647307937804j,
                 -0.786455213599086 - 0.617647307937804j,
                 0.617647307937804 - 0.786455213599086j,
                 -0.617647307937804 + 0.786455213599086j,
                 0.962953266873684 + 0.269668325572915j,
                 -0.962953266873684 - 0.269668325572915j,
                 0.269668325572915 - 0.962953266873684j,
                 -0.269668325572915 + 0.962953266873684j,
                 0.871595086655951 - 0.490226483288291j,
                 -0.871595086655951 + 0.490226483288291j,
                 0.490226483288291 + 0.871595086655951j,
                 -0.490226483288291 - 0.871595086655951j,
                 0.999618822495179 + 0.0276081457789657j,
                 -0.999618822495179 - 0.0276081457789657j,
                 0.0276081457789657 - 0.999618822495179j,
                 -0.0276081457789657 + 0.999618822495179j,
                 0.726359155084346 - 0.687315340891759j,
                 -0.726359155084346 + 0.687315340891759j,
                 0.687315340891759 + 0.726359155084346j,
                 -0.687315340891759 - 0.726359155084346j,
                 0.934092550404259 - 0.357030961233430j,
                 -0.934092550404259 + 0.357030961233430j,
                 0.357030961233430 + 0.934092550404259j,
                 -0.357030961233430 - 0.934092550404259j,
                 0.912962190428398 + 0.408044162864979j,
                 -0.912962190428398 - 0.408044162864979j,
                 0.408044162864979 - 0.912962190428398j,
                 -0.408044162864979 + 0.912962190428398j,
                 0.985797509167567 - 0.167938294974731j,
                 -0.985797509167567 + 0.167938294974731j,
                 0.167938294974731 + 0.985797509167567j,
                 -0.167938294974731 - 0.985797509167567j,
                 0.815814410806734 + 0.578313796411656j,
                 -0.815814410806734 - 0.578313796411656j,
                 0.578313796411656 - 0.815814410806734j,
                 -0.578313796411656 + 0.815814410806734j,
                 0.975025345066994 + 0.222093620973204j,
                 -0.975025345066994 - 0.222093620973204j,
                 0.222093620973204 - 0.975025345066994j,
                 -0.222093620973204 + 0.975025345066994j,
                 0.846490938774052 - 0.532403127877198j,
                 -0.846490938774052 + 0.532403127877198j,
                 0.532403127877198 + 0.846490938774052j,
                 -0.532403127877198 - 0.846490938774052j,
                 0.997511456140303 - 0.0705045733896139j,
                 -0.997511456140303 + 0.0705045733896139j,
                 0.0705045733896139 + 0.997511456140303j,
                 -0.0705045733896139 - 0.997511456140303j,
                 0.755201376896537 + 0.655492852999615j,
                 -0.755201376896537 - 0.655492852999615j,
                 0.655492852999615 - 0.755201376896537j,
                 -0.655492852999615 + 0.755201376896537j,
                 0.948561349915730 + 0.316593375556166j,
                 -0.948561349915730 - 0.316593375556166j,
                 0.316593375556166 - 0.948561349915730j,
                 -0.316593375556166 + 0.948561349915730j,
                 0.894599485631383 - 0.446868840162374j,
                 -0.894599485631383 + 0.446868840162374j,
                 0.446868840162374 + 0.894599485631383j,
                 -0.446868840162374 - 0.894599485631383j,
                 0.992099313142192 + 0.125454983411546j,
                 -0.992099313142192 - 0.125454983411546j,
                 0.125454983411546 - 0.992099313142192j,
                 -0.125454983411546 + 0.992099313142192j,
                 0.790230221437310 - 0.612810082429410j,
                 -0.790230221437310 + 0.612810082429410j,
                 0.612810082429410 + 0.790230221437310j,
                 -0.612810082429410 - 0.790230221437310j,
                 0.964589793289813 - 0.263754678974831j,
                 -0.964589793289813 + 0.263754678974831j,
                 0.263754678974831 + 0.964589793289813j,
                 -0.263754678974831 - 0.964589793289813j,
                 0.868570705971341 + 0.495565261825772j,
                 -0.868570705971341 - 0.495565261825772j,
                 0.495565261825772 - 0.868570705971341j,
                 -0.495565261825772 + 0.868570705971341j,
                 0.999957644551964 - 0.00920375478205982j,
                 -0.999957644551964 + 0.00920375478205982j,
                 0.00920375478205982 + 0.999957644551964j,
                 -0.00920375478205982 - 0.999957644551964j,
                 0.713584868780794 + 0.700568793943248j,
                 -0.713584868780794 - 0.700568793943248j,
                 0.700568793943248 - 0.713584868780794j,
                 -0.700568793943248 + 0.713584868780794j,
                 0.927362525650401 + 0.374164062971458j,
                 -0.927362525650401 - 0.374164062971458j,
                 0.374164062971458 - 0.927362525650401j,
                 -0.374164062971458 + 0.927362525650401j,
                 0.920318276709110 - 0.391170384302254j,
                 -0.920318276709110 + 0.391170384302254j,
                 0.391170384302254 + 0.920318276709111j,
                 -0.391170384302254 - 0.920318276709111j,
                 0.982539302287441 + 0.186055151663447j,
                 -0.982539302287441 - 0.186055151663447j,
                 0.186055151663447 - 0.982539302287441j,
                 -0.186055151663447 + 0.982539302287441j,
                 0.826321062845664 - 0.563199344013834j,
                 -0.826321062845664 + 0.563199344013834j,
                 0.563199344013834 + 0.826321062845664j,
                 -0.563199344013834 - 0.826321062845664j,
                 0.978948175319062 - 0.204108966092817j,
                 -0.978948175319062 + 0.204108966092817j,
                 0.204108966092817 + 0.978948175319062j,
                 -0.204108966092817 - 0.978948175319062j,
                 0.836547727223512 + 0.547894059173100j,
                 -0.836547727223512 - 0.547894059173100j,
                 0.547894059173100 - 0.836547727223512j,
                 -0.547894059173100 + 0.836547727223512j,
                 0.996044700901252 + 0.0888535525825246j,
                 -0.996044700901252 - 0.0888535525825246j,
                 0.0888535525825246 - 0.996044700901252j,
                 -0.0888535525825246 + 0.996044700901252j,
                 0.767138911935820 - 0.641481012808583j,
                 -0.767138911935820 + 0.641481012808583j,
                 0.641481012808583 + 0.767138911935820j,
                 -0.641481012808583 - 0.767138911935820j,
                 0.954228095109106 - 0.299079826308040j,
                 -0.954228095109106 + 0.299079826308040j,
                 0.299079826308040 + 0.954228095109106j,
                 -0.299079826308040 - 0.954228095109106j,
                 0.886222530148881 + 0.463259783551860j,
                 -0.886222530148881 - 0.463259783551860j,
                 0.463259783551860 - 0.886222530148881j,
                 -0.463259783551860 + 0.886222530148881j,
                 0.994240449453188 - 0.107172424956809j,
                 -0.994240449453188 + 0.107172424956809j,
                 0.107172424956809 + 0.994240449453188j,
                 -0.107172424956809 - 0.994240449453188j,
                 0.778816512381476 + 0.627251815495144j,
                 -0.778816512381476 - 0.627251815495144j,
                 0.627251815495144 - 0.778816512381476j,
                 -0.627251815495144 + 0.778816512381476j,
                 0.959571513081985 + 0.281464937925758j,
                 -0.959571513081985 - 0.281464937925758j,
                 0.281464937925758 - 0.959571513081985j,
                 -0.281464937925758 + 0.959571513081985j,
                 0.877545290207261 - 0.479493757660153j,
                 -0.877545290207261 + 0.479493757660153j,
                 0.479493757660153 + 0.877545290207261j,
                 -0.479493757660153 - 0.877545290207261j,
                 0.999204758618364 + 0.0398729275877398j,
                 -0.999204758618364 - 0.0398729275877398j,
                 0.0398729275877398 - 0.999204758618364j,
                 -0.0398729275877398 + 0.999204758618364j,
                 0.734738878095963 - 0.678350043129862j,
                 -0.734738878095963 + 0.678350043129862j,
                 0.678350043129861 + 0.734738878095963j,
                 -0.678350043129861 - 0.734738878095963j,
                 0.938403534063108 - 0.345541324963989j,
                 -0.938403534063108 + 0.345541324963989j,
                 0.345541324963989 + 0.938403534063108j,
                 -0.345541324963989 - 0.938403534063108j,
                 0.907886116487666 + 0.419216888363224j,
                 -0.907886116487666 - 0.419216888363224j,
                 0.419216888363224 - 0.907886116487666j,
                 -0.419216888363224 + 0.907886116487666j,
                 0.987784141644572 - 0.155828397654265j,
                 -0.987784141644572 + 0.155828397654265j,
                 0.155828397654265 + 0.987784141644572j,
                 -0.155828397654265 - 0.987784141644572j,
                 0.808656181588175 + 0.588281548222645j,
                 -0.808656181588175 - 0.588281548222645j,
                 0.588281548222645 - 0.808656181588175j,
                 -0.588281548222645 + 0.808656181588175j,
                 0.972226497078936 + 0.234041958583543j,
                 -0.972226497078936 - 0.234041958583543j,
                 0.234041958583543 - 0.972226497078936j,
                 -0.234041958583543 + 0.972226497078936j,
                 0.852960604930364 - 0.521975292937154j,
                 -0.852960604930364 + 0.521975292937154j,
                 0.521975292937154 + 0.852960604930364j,
                 -0.521975292937154 - 0.852960604930364j,
                 0.998301544933893 - 0.0582582645004358j,
                 -0.998301544933893 + 0.0582582645004358j,
                 0.0582582645004358 + 0.998301544933893j,
                 -0.0582582645004358 - 0.998301544933893j,
                 0.747100605980180 + 0.664710978203345j,
                 -0.747100605980180 - 0.664710978203345j,
                 0.664710978203345 - 0.747100605980180j,
                 -0.664710978203345 + 0.747100605980180j,
                 0.944604837261480 + 0.328209843579092j,
                 -0.944604837261480 - 0.328209843579092j,
                 0.328209843579092 - 0.944604837261480j,
                 -0.328209843579092 + 0.944604837261480j,
                 0.900015892016160 - 0.435857079922255j,
                 -0.900015892016160 + 0.435857079922255j,
                 0.435857079922256 + 0.900015892016160j,
                 -0.435857079922256 - 0.900015892016160j,
                 0.990485084256457 + 0.137620121586486j,
                 -0.990485084256457 - 0.137620121586486j,
                 0.137620121586486 - 0.990485084256457j,
                 -0.137620121586486 + 0.990485084256457j,
                 0.797690840943391 - 0.603066598540348j,
                 -0.797690840943391 + 0.603066598540348j,
                 0.603066598540348 + 0.797690840943391j,
                 -0.603066598540348 - 0.797690840943391j,
                 0.967753837093475 - 0.251897818154217j,
                 -0.967753837093475 + 0.251897818154217j,
                 0.251897818154217 + 0.967753837093475j,
                 -0.251897818154217 - 0.967753837093475j,
                 0.862423956111041 + 0.506186645345155j,
                 -0.862423956111041 - 0.506186645345155j,
                 0.506186645345155 - 0.862423956111041j,
                 -0.506186645345155 + 0.862423956111041j,
                 0.999882347454213 + 0.0153392062849881j,
                 -0.999882347454213 - 0.0153392062849881j,
                 0.0153392062849881 - 0.999882347454213j,
                 -0.0153392062849881 + 0.999882347454213j,
                 0.717870045055732 - 0.696177131491463j,
                 -0.717870045055732 + 0.696177131491463j,
                 0.696177131491463 + 0.717870045055732j,
                 -0.696177131491463 - 0.717870045055732j,
                 0.929640895843181 - 0.368466829953372j,
                 -0.929640895843181 + 0.368466829953372j,
                 0.368466829953372 + 0.929640895843181j,
                 -0.368466829953372 - 0.929640895843181j,
                 0.917900775621390 + 0.396809987416710j,
                 -0.917900775621390 - 0.396809987416710j,
                 0.396809987416710 - 0.917900775621390j,
                 -0.396809987416710 + 0.917900775621390j,
                 0.983662419211730 - 0.180022901405700j,
                 -0.983662419211730 + 0.180022901405700j,
                 0.180022901405700 + 0.983662419211730j,
                 -0.180022901405700 - 0.983662419211730j,
                 0.822849781375826 + 0.568258952670132j,
                 -0.822849781375826 - 0.568258952670132j,
                 0.568258952670132 - 0.822849781375826j,
                 -0.568258952670132 + 0.822849781375826j,
                 0.977677357824510 + 0.210111836880470j,
                 -0.977677357824510 - 0.210111836880470j,
                 0.210111836880470 - 0.977677357824510j,
                 -0.210111836880470 + 0.977677357824510j,
                 0.839893794196000 - 0.542750784864516j,
                 -0.839893794196000 + 0.542750784864516j,
                 0.542750784864516 + 0.839893794196000j,
                 -0.542750784864516 - 0.839893794196000j,
                 0.996571145790555 - 0.0827402645493757j,
                 -0.996571145790555 + 0.0827402645493757j,
                 0.0827402645493757 + 0.996571145790555j,
                 -0.0827402645493757 - 0.996571145790555j,
                 0.763188417263381 + 0.646176012983316j,
                 -0.763188417263381 - 0.646176012983316j,
                 0.646176012983316 - 0.763188417263381j,
                 -0.646176012983316 + 0.763188417263381j,
                 0.952375012719766 + 0.304929229735402j,
                 -0.952375012719766 - 0.304929229735402j,
                 0.304929229735402 - 0.952375012719766j,
                 -0.304929229735402 + 0.952375012719766j,
                 0.889048355854665 - 0.457813303598877j,
                 -0.889048355854665 + 0.457813303598877j,
                 0.457813303598877 + 0.889048355854665j,
                 -0.457813303598877 - 0.889048355854665j,
                 0.993564135520595 + 0.113270952177564j,
                 -0.993564135520595 - 0.113270952177564j,
                 0.113270952177564 - 0.993564135520595j,
                 -0.113270952177564 + 0.993564135520595j,
                 0.782650596166576 - 0.622461279374150j,
                 -0.782650596166576 + 0.622461279374150j,
                 0.622461279374150 + 0.782650596166576j,
                 -0.622461279374150 - 0.782650596166576j,
                 0.961280485811321 - 0.275571819310958j,
                 -0.961280485811321 + 0.275571819310958j,
                 0.275571819310958 + 0.961280485811321j,
                 -0.275571819310958 - 0.961280485811321j,
                 0.874586652278176 + 0.484869248000791j,
                 -0.874586652278176 - 0.484869248000791j,
                 0.484869248000791 - 0.874586652278176j,
                 -0.484869248000791 + 0.874586652278176j,
                 0.999430604555462 - 0.0337411718513776j,
                 -0.999430604555462 + 0.0337411718513776j,
                 0.0337411718513776 + 0.999430604555462j,
                 -0.0337411718513776 - 0.999430604555462j,
                 0.730562769227828 + 0.682845546385248j,
                 -0.730562769227828 - 0.682845546385248j,
                 0.682845546385248 - 0.730562769227828j,
                 -0.682845546385248 + 0.730562769227828j,
                 0.936265667170278 + 0.351292756085567j,
                 -0.936265667170278 - 0.351292756085567j,
                 0.351292756085567 - 0.936265667170278j,
                 -0.351292756085567 + 0.936265667170278j,
                 0.910441292258067 - 0.413638312238435j,
                 -0.910441292258067 + 0.413638312238435j,
                 0.413638312238435 + 0.910441292258067j,
                 -0.413638312238435 - 0.910441292258067j,
                 0.986809401814185 + 0.161886393780112j,
                 -0.986809401814185 - 0.161886393780112j,
                 0.161886393780112 - 0.986809401814185j,
                 -0.161886393780112 + 0.986809401814185j,
                 0.812250586585204 - 0.583308652937698j,
                 -0.812250586585204 + 0.583308652937698j,
                 0.583308652937698 + 0.812250586585204j,
                 -0.583308652937698 - 0.812250586585204j,
                 0.973644249650812 - 0.228072083170886j,
                 -0.973644249650812 + 0.228072083170886j,
                 0.228072083170886 + 0.973644249650812j,
                 -0.228072083170886 - 0.973644249650812j,
                 0.849741768000852 + 0.527199134781901j,
                 -0.849741768000852 - 0.527199134781901j,
                 0.527199134781901 - 0.849741768000853j,
                 -0.527199134781901 + 0.849741768000853j,
                 0.997925286198596 + 0.0643826309298575j,
                 -0.997925286198596 - 0.0643826309298575j,
                 0.0643826309298575 - 0.997925286198596j,
                 -0.0643826309298575 + 0.997925286198596j,
                 0.751165131909686 - 0.660114342067420j,
                 -0.751165131909686 + 0.660114342067420j,
                 0.660114342067420 + 0.751165131909686j,
                 -0.660114342067420 - 0.751165131909686j,
                 0.946600913083284 - 0.322407678801070j,
                 -0.946600913083284 + 0.322407678801070j,
                 0.322407678801070 + 0.946600913083284j,
                 -0.322407678801070 - 0.946600913083284j,
                 0.897324580705418 + 0.441371268731717j,
                 -0.897324580705418 - 0.441371268731717j,
                 0.441371268731717 - 0.897324580705418j,
                 -0.441371268731717 + 0.897324580705418j,
                 0.991310859846115 - 0.131540028702883j,
                 -0.991310859846115 + 0.131540028702883j,
                 0.131540028702883 + 0.991310859846115j,
                 -0.131540028702883 - 0.991310859846115j,
                 0.793975477554337 + 0.607949784967774j,
                 -0.793975477554337 - 0.607949784967774j,
                 0.607949784967774 - 0.793975477554337j,
                 -0.607949784967774 + 0.793975477554337j,
                 0.966190003445413 + 0.257831102162159j,
                 -0.966190003445413 - 0.257831102162159j,
                 0.257831102162159 - 0.966190003445413j,
                 -0.257831102162159 + 0.966190003445413j,
                 0.865513624090569 - 0.500885382611241j,
                 -0.865513624090569 + 0.500885382611241j,
                 0.500885382611241 + 0.865513624090569j,
                 -0.500885382611241 - 0.865513624090569j,
                 0.999995293809576 - 0.00306795676296598j,
                 -0.999995293809576 + 0.00306795676296598j,
                 0.00306795676296598 + 0.999995293809576j,
                 -0.00306795676296598 - 0.999995293809576j,
                 0.709272826438866 + 0.704934080375905j,
                 -0.709272826438866 - 0.704934080375905j,
                 0.704934080375905 - 0.709272826438866j,
                 -0.704934080375905 + 0.709272826438866j,
                 0.925049240782678 + 0.379847208924051j,
                 -0.925049240782678 - 0.379847208924051j,
                 0.379847208924051 - 0.925049240782678j,
                 -0.379847208924051 + 0.925049240782678j,
                 0.922701128333879 - 0.385516053843919j,
                 -0.922701128333879 + 0.385516053843919j,
                 0.385516053843919 + 0.922701128333879j,
                 -0.385516053843919 - 0.922701128333879j,
                 0.981379193313755 + 0.192080397049892j,
                 -0.981379193313755 - 0.192080397049892j,
                 0.192080397049892 - 0.981379193313755j,
                 -0.192080397049892 + 0.981379193313755j,
                 0.829761233794523 - 0.558118531220556j,
                 -0.829761233794523 + 0.558118531220556j,
                 0.558118531220556 + 0.829761233794523j,
                 -0.558118531220556 - 0.829761233794523j,
                 0.980182135968117 - 0.198098410717954j,
                 -0.980182135968117 + 0.198098410717954j,
                 0.198098410717954 + 0.980182135968117j,
                 -0.198098410717954 - 0.980182135968117j,
                 0.833170164701913 + 0.553016705580028j,
                 -0.833170164701913 - 0.553016705580028j,
                 0.553016705580028 - 0.833170164701913j,
                 -0.553016705580028 + 0.833170164701913j,
                 0.995480755491927 + 0.0949634953296390j,
                 -0.995480755491927 - 0.0949634953296390j,
                 0.0949634953296390 - 0.995480755491927j,
                 -0.0949634953296390 + 0.995480755491927j,
                 0.771060524261814 - 0.636761861236284j,
                 -0.771060524261814 + 0.636761861236284j,
                 0.636761861236284 + 0.771060524261814j,
                 -0.636761861236284 - 0.771060524261814j,
                 0.956045251349996 - 0.293219162694259j,
                 -0.956045251349996 + 0.293219162694259j,
                 0.293219162694259 + 0.956045251349996j,
                 -0.293219162694259 - 0.956045251349996j,
                 0.883363338665732 + 0.468688822035828j,
                 -0.883363338665732 - 0.468688822035828j,
                 0.468688822035828 - 0.883363338665732j,
                 -0.468688822035828 + 0.883363338665732j,
                 0.994879330794806 - 0.101069862754828j,
                 -0.994879330794806 + 0.101069862754828j,
                 0.101069862754828 + 0.994879330794806j,
                 -0.101069862754828 - 0.994879330794806j,
                 0.774953106594874 + 0.632018735939809j,
                 -0.774953106594874 - 0.632018735939809j,
                 0.632018735939809 - 0.774953106594874j,
                 -0.632018735939809 + 0.774953106594874j,
                 0.957826413027533 + 0.287347459544730j,
                 -0.957826413027533 - 0.287347459544730j,
                 0.287347459544730 - 0.957826413027533j,
                 -0.287347459544730 + 0.957826413027533j,
                 0.880470889052161 - 0.474100214650550j,
                 -0.880470889052161 + 0.474100214650550j,
                 0.474100214650550 + 0.880470889052161j,
                 -0.474100214650550 - 0.880470889052161j,
                 0.998941293186857 + 0.0460031821309146j,
                 -0.998941293186857 - 0.0460031821309146j,
                 0.0460031821309146 - 0.998941293186857j,
                 -0.0460031821309146 + 0.998941293186857j,
                 0.738887324460615 - 0.673829000378756j,
                 -0.738887324460615 + 0.673829000378756j,
                 0.673829000378756 + 0.738887324460615j,
                 -0.673829000378756 - 0.738887324460615j,
                 0.940506070593268 - 0.339776884406827j,
                 -0.940506070593268 + 0.339776884406827j,
                 0.339776884406827 + 0.940506070593268j,
                 -0.339776884406827 - 0.940506070593268j,
                 0.905296759318119 + 0.424779681209109j,
                 -0.905296759318119 - 0.424779681209109j,
                 0.424779681209109 - 0.905296759318119j,
                 -0.424779681209109 + 0.905296759318119j,
                 0.988721691960324 - 0.149764534677322j,
                 -0.988721691960324 + 0.149764534677322j,
                 0.149764534677322 + 0.988721691960324j,
                 -0.149764534677322 - 0.988721691960324j,
                 0.805031331142964 + 0.593232295039800j,
                 -0.805031331142964 - 0.593232295039800j,
                 0.593232295039800 - 0.805031331142964j,
                 -0.593232295039800 + 0.805031331142964j,
                 0.970772140728950 + 0.240003022448741j,
                 -0.970772140728950 - 0.240003022448741j,
                 0.240003022448741 - 0.970772140728950j,
                 -0.240003022448741 + 0.970772140728950j,
                 0.856147328375194 - 0.516731799017650j,
                 -0.856147328375194 + 0.516731799017650j,
                 0.516731799017650 + 0.856147328375194j,
                 -0.516731799017650 - 0.856147328375194j,
                 0.998640218180265 - 0.0521317046802833j,
                 -0.998640218180265 + 0.0521317046802833j,
                 0.0521317046802833 + 0.998640218180265j,
                 -0.0521317046802833 - 0.998640218180265j,
                 0.743007952135122 + 0.669282588346636j,
                 -0.743007952135122 - 0.669282588346636j,
                 0.669282588346636 - 0.743007952135122j,
                 -0.669282588346636 + 0.743007952135122j,
                 0.942573197601447 + 0.333999651442009j,
                 -0.942573197601447 - 0.333999651442009j,
                 0.333999651442009 - 0.942573197601447j,
                 -0.333999651442009 + 0.942573197601447j,
                 0.902673318237259 - 0.430326481340083j,
                 -0.902673318237259 + 0.430326481340083j,
                 0.430326481340083 + 0.902673318237259j,
                 -0.430326481340083 - 0.902673318237259j,
                 0.989622017463201 + 0.143695033150294j,
                 -0.989622017463201 - 0.143695033150294j,
                 0.143695033150294 - 0.989622017463201j,
                 -0.143695033150294 + 0.989622017463201j,
                 0.801376171723140 - 0.598160706996342j,
                 -0.801376171723140 + 0.598160706996342j,
                 0.598160706996342 + 0.801376171723140j,
                 -0.598160706996342 - 0.801376171723140j,
                 0.969281235356549 - 0.245955050335795j,
                 -0.969281235356549 + 0.245955050335795j,
                 0.245955050335795 + 0.969281235356549j,
                 -0.245955050335795 - 0.969281235356549j,
                 0.859301818357008 + 0.511468850437970j,
                 -0.859301818357008 - 0.511468850437970j,
                 0.511468850437970 - 0.859301818357008j,
                 -0.511468850437970 + 0.859301818357008j,
                 0.999769405351215 + 0.0214740802754695j,
                 -0.999769405351215 - 0.0214740802754695j,
                 0.0214740802754695 - 0.999769405351215j,
                 -0.0214740802754695 + 0.999769405351215j,
                 0.722128193929215 - 0.691759258364158j,
                 -0.722128193929215 + 0.691759258364158j,
                 0.691759258364158 + 0.722128193929215j,
                 -0.691759258364158 - 0.722128193929215j,
                 0.931884265581668 - 0.362755724367397j,
                 -0.931884265581668 + 0.362755724367397j,
                 0.362755724367397 + 0.931884265581668j,
                 -0.362755724367397 - 0.931884265581668j,
                 0.915448716088268 + 0.402434650859418j,
                 -0.915448716088268 - 0.402434650859418j,
                 0.402434650859418 - 0.915448716088268j,
                 -0.402434650859418 + 0.915448716088268j,
                 0.984748501801904 - 0.173983873387464j,
                 -0.984748501801904 + 0.173983873387464j,
                 0.173983873387464 + 0.984748501801904j,
                 -0.173983873387464 - 0.984748501801904j,
                 0.819347520076797 + 0.573297166698042j,
                 -0.819347520076797 - 0.573297166698042j,
                 0.573297166698042 - 0.819347520076797j,
                 -0.573297166698042 + 0.819347520076797j,
                 0.976369731330021 + 0.216106797076220j,
                 -0.976369731330021 - 0.216106797076220j,
                 0.216106797076220 - 0.976369731330021j,
                 -0.216106797076220 + 0.976369731330021j,
                 0.843208239641845 - 0.537587076295646j,
                 -0.843208239641845 + 0.537587076295646j,
                 0.537587076295646 + 0.843208239641845j,
                 -0.537587076295646 - 0.843208239641845j,
                 0.997060070339483 - 0.0766238613920315j,
                 -0.997060070339483 + 0.0766238613920315j,
                 0.0766238613920315 + 0.997060070339483j,
                 -0.0766238613920315 - 0.997060070339483j,
                 0.759209188978388 + 0.650846684996381j,
                 -0.759209188978388 - 0.650846684996381j,
                 0.650846684996381 - 0.759209188978388j,
                 -0.650846684996381 + 0.759209188978388j,
                 0.950486073949482 + 0.310767152749612j,
                 -0.950486073949482 - 0.310767152749612j,
                 0.310767152749612 - 0.950486073949482j,
                 -0.310767152749612 + 0.950486073949482j,
                 0.891840709392343 - 0.452349587233771j,
                 -0.891840709392343 + 0.452349587233771j,
                 0.452349587233771 + 0.891840709392343j,
                 -0.452349587233771 - 0.891840709392343j,
                 0.992850414459865 + 0.119365214810991j,
                 -0.992850414459865 - 0.119365214810991j,
                 0.119365214810991 - 0.992850414459865j,
                 -0.119365214810991 + 0.992850414459865j,
                 0.786455213599086 - 0.617647307937804j,
                 -0.786455213599086 + 0.617647307937804j,
                 0.617647307937804 + 0.786455213599086j,
                 -0.617647307937804 - 0.786455213599086j,
                 0.962953266873684 - 0.269668325572915j,
                 -0.962953266873684 + 0.269668325572915j,
                 0.269668325572915 + 0.962953266873684j,
                 -0.269668325572915 - 0.962953266873684j,
                 0.871595086655951 + 0.490226483288291j,
                 -0.871595086655951 - 0.490226483288291j,
                 0.490226483288291 - 0.871595086655951j,
                 -0.490226483288291 + 0.871595086655951j,
                 0.999618822495179 - 0.0276081457789657j,
                 -0.999618822495179 + 0.0276081457789657j,
                 0.0276081457789657 + 0.999618822495179j,
                 -0.0276081457789657 - 0.999618822495179j,
                 0.726359155084346 + 0.687315340891759j,
                 -0.726359155084346 - 0.687315340891759j,
                 0.687315340891759 - 0.726359155084346j,
                 -0.687315340891759 + 0.726359155084346j,
                 0.934092550404259 + 0.357030961233430j,
                 -0.934092550404259 - 0.357030961233430j,
                 0.357030961233430 - 0.934092550404259j,
                 -0.357030961233430 + 0.934092550404259j,
                 0.912962190428398 - 0.408044162864979j,
                 -0.912962190428398 + 0.408044162864979j,
                 0.408044162864979 + 0.912962190428398j,
                 -0.408044162864979 - 0.912962190428398j,
                 0.985797509167567 + 0.167938294974731j,
                 -0.985797509167567 - 0.167938294974731j,
                 0.167938294974731 - 0.985797509167567j,
                 -0.167938294974731 + 0.985797509167567j,
                 0.815814410806734 - 0.578313796411656j,
                 -0.815814410806734 + 0.578313796411656j,
                 0.578313796411656 + 0.815814410806734j,
                 -0.578313796411656 - 0.815814410806734j,
                 0.975025345066994 - 0.222093620973204j,
                 -0.975025345066994 + 0.222093620973204j,
                 0.222093620973204 + 0.975025345066994j,
                 -0.222093620973204 - 0.975025345066994j,
                 0.846490938774052 + 0.532403127877198j,
                 -0.846490938774052 - 0.532403127877198j,
                 0.532403127877198 - 0.846490938774052j,
                 -0.532403127877198 + 0.846490938774052j,
                 0.997511456140303 + 0.0705045733896139j,
                 -0.997511456140303 - 0.0705045733896139j,
                 0.0705045733896139 - 0.997511456140303j,
                 -0.0705045733896139 + 0.997511456140303j,
                 0.755201376896537 - 0.655492852999615j,
                 -0.755201376896537 + 0.655492852999615j,
                 0.655492852999615 + 0.755201376896537j,
                 -0.655492852999615 - 0.755201376896537j,
                 0.948561349915730 - 0.316593375556166j,
                 -0.948561349915730 + 0.316593375556166j,
                 0.316593375556166 + 0.948561349915730j,
                 -0.316593375556166 - 0.948561349915730j,
                 0.894599485631383 + 0.446868840162374j,
                 -0.894599485631383 - 0.446868840162374j,
                 0.446868840162374 - 0.894599485631383j,
                 -0.446868840162374 + 0.894599485631383j,
                 0.992099313142192 - 0.125454983411546j,
                 -0.992099313142192 + 0.125454983411546j,
                 0.125454983411546 + 0.992099313142192j,
                 -0.125454983411546 - 0.992099313142192j,
                 0.790230221437310 + 0.612810082429410j,
                 -0.790230221437310 - 0.612810082429410j,
                 0.612810082429410 - 0.790230221437310j,
                 -0.612810082429410 + 0.790230221437310j,
                 0.964589793289813 + 0.263754678974831j,
                 -0.964589793289813 - 0.263754678974831j,
                 0.263754678974831 - 0.964589793289813j,
                 -0.263754678974831 + 0.964589793289813j,
                 0.868570705971341 - 0.495565261825772j,
                 -0.868570705971341 + 0.495565261825772j,
                 0.495565261825772 + 0.868570705971341j,
                 -0.495565261825772 - 0.868570705971341j,
                 0.999957644551964 + 0.00920375478205982j,
                 -0.999957644551964 - 0.00920375478205982j,
                 0.00920375478205982 - 0.999957644551964j,
                 -0.00920375478205982 + 0.999957644551964j,
                 0.713584868780794 - 0.700568793943248j,
                 -0.713584868780794 + 0.700568793943248j,
                 0.700568793943248 + 0.713584868780794j,
                 -0.700568793943248 - 0.713584868780794j,
                 0.927362525650401 - 0.374164062971458j,
                 -0.927362525650401 + 0.374164062971458j,
                 0.374164062971458 + 0.927362525650401j,
                 -0.374164062971458 - 0.927362525650401j,
                 0.920318276709110 + 0.391170384302254j,
                 -0.920318276709110 - 0.391170384302254j,
                 0.391170384302254 - 0.920318276709111j,
                 -0.391170384302254 + 0.920318276709111j,
                 0.982539302287441 - 0.186055151663447j,
                 -0.982539302287441 + 0.186055151663447j,
                 0.186055151663447 + 0.982539302287441j,
                 -0.186055151663447 - 0.982539302287441j,
                 0.826321062845664 + 0.563199344013834j,
                 -0.826321062845664 - 0.563199344013834j,
                 0.563199344013834 - 0.826321062845664j,
                 -0.563199344013834 + 0.826321062845664j,
                 0.978948175319062 + 0.204108966092817j,
                 -0.978948175319062 - 0.204108966092817j,
                 0.204108966092817 - 0.978948175319062j,
                 -0.204108966092817 + 0.978948175319062j,
                 0.836547727223512 - 0.547894059173100j,
                 -0.836547727223512 + 0.547894059173100j,
                 0.547894059173100 + 0.836547727223512j,
                 -0.547894059173100 - 0.836547727223512j,
                 0.996044700901252 - 0.0888535525825246j,
                 -0.996044700901252 + 0.0888535525825246j,
                 0.0888535525825246 + 0.996044700901252j,
                 -0.0888535525825246 - 0.996044700901252j,
                 0.767138911935820 + 0.641481012808583j,
                 -0.767138911935820 - 0.641481012808583j,
                 0.641481012808583 - 0.767138911935820j,
                 -0.641481012808583 + 0.767138911935820j,
                 0.954228095109106 + 0.299079826308040j,
                 -0.954228095109106 - 0.299079826308040j,
                 0.299079826308040 - 0.954228095109106j,
                 -0.299079826308040 + 0.954228095109106j,
                 0.886222530148881 - 0.463259783551860j,
                 -0.886222530148881 + 0.463259783551860j,
                 0.463259783551860 + 0.886222530148881j,
                 -0.463259783551860 - 0.886222530148881j,
                 0.994240449453188 + 0.107172424956809j,
                 -0.994240449453188 - 0.107172424956809j,
                 0.107172424956809 - 0.994240449453188j,
                 -0.107172424956809 + 0.994240449453188j,
                 0.778816512381476 - 0.627251815495144j,
                 -0.778816512381476 + 0.627251815495144j,
                 0.627251815495144 + 0.778816512381476j,
                 -0.627251815495144 - 0.778816512381476j,
                 0.959571513081985 - 0.281464937925758j,
                 -0.959571513081985 + 0.281464937925758j,
                 0.281464937925758 + 0.959571513081985j,
                 -0.281464937925758 - 0.959571513081985j,
                 0.877545290207261 + 0.479493757660153j,
                 -0.877545290207261 - 0.479493757660153j,
                 0.479493757660153 - 0.877545290207261j,
                 -0.479493757660153 + 0.877545290207261j,
                 0.999204758618364 - 0.0398729275877398j,
                 -0.999204758618364 + 0.0398729275877398j,
                 0.0398729275877398 + 0.999204758618364j,
                 -0.0398729275877398 - 0.999204758618364j,
                 0.734738878095963 + 0.678350043129862j,
                 -0.734738878095963 - 0.678350043129862j,
                 0.678350043129861 - 0.734738878095963j,
                 -0.678350043129861 + 0.734738878095963j,
                 0.938403534063108 + 0.345541324963989j,
                 -0.938403534063108 - 0.345541324963989j,
                 0.345541324963989 - 0.938403534063108j,
                 -0.345541324963989 + 0.938403534063108j,
                 0.907886116487666 - 0.419216888363224j,
                 -0.907886116487666 + 0.419216888363224j,
                 0.419216888363224 + 0.907886116487666j,
                 -0.419216888363224 - 0.907886116487666j,
                 0.987784141644572 + 0.155828397654265j,
                 -0.987784141644572 - 0.155828397654265j,
                 0.155828397654265 - 0.987784141644572j,
                 -0.155828397654265 + 0.987784141644572j,
                 0.808656181588175 - 0.588281548222645j,
                 -0.808656181588175 + 0.588281548222645j,
                 0.588281548222645 + 0.808656181588175j,
                 -0.588281548222645 - 0.808656181588175j,
                 0.972226497078936 - 0.234041958583543j,
                 -0.972226497078936 + 0.234041958583543j,
                 0.234041958583543 + 0.972226497078936j,
                 -0.234041958583543 - 0.972226497078936j,
                 0.852960604930364 + 0.521975292937154j,
                 -0.852960604930364 - 0.521975292937154j,
                 0.521975292937154 - 0.852960604930364j,
                 -0.521975292937154 + 0.852960604930364j,
                 0.998301544933893 + 0.0582582645004358j,
                 -0.998301544933893 - 0.0582582645004358j,
                 0.0582582645004358 - 0.998301544933893j,
                 -0.0582582645004358 + 0.998301544933893j,
                 0.747100605980180 - 0.664710978203345j,
                 -0.747100605980180 + 0.664710978203345j,
                 0.664710978203345 + 0.747100605980180j,
                 -0.664710978203345 - 0.747100605980180j,
                 0.944604837261480 - 0.328209843579092j,
                 -0.944604837261480 + 0.328209843579092j,
                 0.328209843579092 + 0.944604837261480j,
                 -0.328209843579092 - 0.944604837261480j,
                 0.900015892016160 + 0.435857079922255j,
                 -0.900015892016160 - 0.435857079922255j,
                 0.435857079922256 - 0.900015892016160j,
                 -0.435857079922256 + 0.900015892016160j,
                 0.990485084256457 - 0.137620121586486j,
                 -0.990485084256457 + 0.137620121586486j,
                 0.137620121586486 + 0.990485084256457j,
                 -0.137620121586486 - 0.990485084256457j,
                 0.797690840943391 + 0.603066598540348j,
                 -0.797690840943391 - 0.603066598540348j,
                 0.603066598540348 - 0.797690840943391j,
                 -0.603066598540348 + 0.797690840943391j,
                 0.967753837093475 + 0.251897818154217j,
                 -0.967753837093475 - 0.251897818154217j,
                 0.251897818154217 - 0.967753837093475j,
                 -0.251897818154217 + 0.967753837093475j,
                 0.862423956111041 - 0.506186645345155j,
                 -0.862423956111041 + 0.506186645345155j,
                 0.506186645345155 + 0.862423956111041j,
                 -0.506186645345155 - 0.862423956111041j,
                 0.999882347454213 - 0.0153392062849881j,
                 -0.999882347454213 + 0.0153392062849881j,
                 0.0153392062849881 + 0.999882347454213j,
                 -0.0153392062849881 - 0.999882347454213j,
                 0.717870045055732 + 0.696177131491463j,
                 -0.717870045055732 - 0.696177131491463j,
                 0.696177131491463 - 0.717870045055732j,
                 -0.696177131491463 + 0.717870045055732j,
                 0.929640895843181 + 0.368466829953372j,
                 -0.929640895843181 - 0.368466829953372j,
                 0.368466829953372 - 0.929640895843181j,
                 -0.368466829953372 + 0.929640895843181j,
                 0.917900775621390 - 0.396809987416710j,
                 -0.917900775621390 + 0.396809987416710j,
                 0.396809987416710 + 0.917900775621390j,
                 -0.396809987416710 - 0.917900775621390j,
                 0.983662419211730 + 0.180022901405700j,
                 -0.983662419211730 - 0.180022901405700j,
                 0.180022901405700 - 0.983662419211730j,
                 -0.180022901405700 + 0.983662419211730j,
                 0.822849781375826 - 0.568258952670132j,
                 -0.822849781375826 + 0.568258952670132j,
                 0.568258952670132 + 0.822849781375826j,
                 -0.568258952670132 - 0.822849781375826j,
                 0.977677357824510 - 0.210111836880470j,
                 -0.977677357824510 + 0.210111836880470j,
                 0.210111836880470 + 0.977677357824510j,
                 -0.210111836880470 - 0.977677357824510j,
                 0.839893794196000 + 0.542750784864516j,
                 -0.839893794196000 - 0.542750784864516j,
                 0.542750784864516 - 0.839893794196000j,
                 -0.542750784864516 + 0.839893794196000j,
                 0.996571145790555 + 0.0827402645493757j,
                 -0.996571145790555 - 0.0827402645493757j,
                 0.0827402645493757 - 0.996571145790555j,
                 -0.0827402645493757 + 0.996571145790555j,
                 0.763188417263381 - 0.646176012983316j,
                 -0.763188417263381 + 0.646176012983316j,
                 0.646176012983316 + 0.763188417263381j,
                 -0.646176012983316 - 0.763188417263381j,
                 0.952375012719766 - 0.304929229735402j,
                 -0.952375012719766 + 0.304929229735402j,
                 0.304929229735402 + 0.952375012719766j,
                 -0.304929229735402 - 0.952375012719766j,
                 0.889048355854665 + 0.457813303598877j,
                 -0.889048355854665 - 0.457813303598877j,
                 0.457813303598877 - 0.889048355854665j,
                 -0.457813303598877 + 0.889048355854665j,
                 0.993564135520595 - 0.113270952177564j,
                 -0.993564135520595 + 0.113270952177564j,
                 0.113270952177564 + 0.993564135520595j,
                 -0.113270952177564 - 0.993564135520595j,
                 0.782650596166576 + 0.622461279374150j,
                 -0.782650596166576 - 0.622461279374150j,
                 0.622461279374150 - 0.782650596166576j,
                 -0.622461279374150 + 0.782650596166576j,
                 0.961280485811321 + 0.275571819310958j,
                 -0.961280485811321 - 0.275571819310958j,
                 0.275571819310958 - 0.961280485811321j,
                 -0.275571819310958 + 0.961280485811321j,
                 0.874586652278176 - 0.484869248000791j,
                 -0.874586652278176 + 0.484869248000791j,
                 0.484869248000791 + 0.874586652278176j,
                 -0.484869248000791 - 0.874586652278176j,
                 0.999430604555462 + 0.0337411718513776j,
                 -0.999430604555462 - 0.0337411718513776j,
                 0.0337411718513776 - 0.999430604555462j,
                 -0.0337411718513776 + 0.999430604555462j,
                 0.730562769227828 - 0.682845546385248j,
                 -0.730562769227828 + 0.682845546385248j,
                 0.682845546385248 + 0.730562769227828j,
                 -0.682845546385248 - 0.730562769227828j,
                 0.936265667170278 - 0.351292756085567j,
                 -0.936265667170278 + 0.351292756085567j,
                 0.351292756085567 + 0.936265667170278j,
                 -0.351292756085567 - 0.936265667170278j,
                 0.910441292258067 + 0.413638312238435j,
                 -0.910441292258067 - 0.413638312238435j,
                 0.413638312238435 - 0.910441292258067j,
                 -0.413638312238435 + 0.910441292258067j,
                 0.986809401814185 - 0.161886393780112j,
                 -0.986809401814185 + 0.161886393780112j,
                 0.161886393780112 + 0.986809401814185j,
                 -0.161886393780112 - 0.986809401814185j,
                 0.812250586585204 + 0.583308652937698j,
                 -0.812250586585204 - 0.583308652937698j,
                 0.583308652937698 - 0.812250586585204j,
                 -0.583308652937698 + 0.812250586585204j,
                 0.973644249650812 + 0.228072083170886j,
                 -0.973644249650812 - 0.228072083170886j,
                 0.228072083170886 - 0.973644249650812j,
                 -0.228072083170886 + 0.973644249650812j,
                 0.849741768000852 - 0.527199134781901j,
                 -0.849741768000852 + 0.527199134781901j,
                 0.527199134781901 + 0.849741768000853j,
                 -0.527199134781901 - 0.849741768000853j,
                 0.997925286198596 - 0.0643826309298575j,
                 -0.997925286198596 + 0.0643826309298575j,
                 0.0643826309298575 + 0.997925286198596j,
                 -0.0643826309298575 - 0.997925286198596j,
                 0.751165131909686 + 0.660114342067420j,
                 -0.751165131909686 - 0.660114342067420j,
                 0.660114342067420 - 0.751165131909686j,
                 -0.660114342067420 + 0.751165131909686j,
                 0.946600913083284 + 0.322407678801070j,
                 -0.946600913083284 - 0.322407678801070j,
                 0.322407678801070 - 0.946600913083284j,
                 -0.322407678801070 + 0.946600913083284j,
                 0.897324580705418 - 0.441371268731717j,
                 -0.897324580705418 + 0.441371268731717j,
                 0.441371268731717 + 0.897324580705418j,
                 -0.441371268731717 - 0.897324580705418j,
                 0.991310859846115 + 0.131540028702883j,
                 -0.991310859846115 - 0.131540028702883j,
                 0.131540028702883 - 0.991310859846115j,
                 -0.131540028702883 + 0.991310859846115j,
                 0.793975477554337 - 0.607949784967774j,
                 -0.793975477554337 + 0.607949784967774j,
                 0.607949784967774 + 0.793975477554337j,
                 -0.607949784967774 - 0.793975477554337j,
                 0.966190003445413 - 0.257831102162159j,
                 -0.966190003445413 + 0.257831102162159j,
                 0.257831102162159 + 0.966190003445413j,
                 -0.257831102162159 - 0.966190003445413j,
                 0.865513624090569 + 0.500885382611241j,
                 -0.865513624090569 - 0.500885382611241j,
                 0.500885382611241 - 0.865513624090569j,
                 -0.500885382611241 + 0.865513624090569j]


"""A dictionary of all the roots."""
roots_dict = {2: phi4_roots,
              4: phi8_roots,
              8: phi16_roots,
              16: phi32_roots,
              32: phi64_roots,
              64: phi128_roots,
              128: phi256_roots,
              256: phi512_roots,
              512: phi1024_roots,
              1024: phi2048_roots,
              1: -1
              }